**Pronostico de la evolución de casos activos de SARS-CoV-2 en Colombia**

El nuevo Coronavirus (COVID-19) ha sido catalogado por la Organización Mundial de la Salud como una emergencia en salud pública de importancia internacional (ESPII). Se han identificado casos en todos los continentes y, el 6 de marzo se confirmó el primer caso en Colombia.De acuerdo con la información aportada por el Instituto Nacional de Salud, para la primera semana del mes de septiembre se ha confirmado alrededor de 640000 casos con una recuperación del alrededor del 75% de la población y una tasa de mortalidad aproximada del 3.2%.

Con el aumento de casos activos en Colombia, se pretende genenerar una herramienta de reporte que permita visualizar predicciones en el corto y mediano plazo del total de casos confirmados, nuevos casos, casos activos, recuperados y muertes para las cinco principales ciudades de Colombia, utilizando técnicas estadísticas, de inteligencia artificial o modelos híbridos.




In [2]:
pip install sodapy

### Importar librerias 

In [3]:
from sodapy import Socrata
import numpy as np 
import re
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import seaborn as sns
import pandas as pd 
import random
import math
import time
import datetime as dt
from datetime import timedelta
import operator 
plt.style.use('fivethirtyeight')
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
#!pip install plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.tsa.stattools import adfuller
!pip install pyramid-arima
from pyramid.arima import auto_arima
std=StandardScaler()


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


     |████████████████████████████████| 604kB 2.7MB/s 


In [4]:
covid_19  = Socrata("www.datos.gov.co", None)
results = covid_19.get_all("gt2j-8ykr", limit=800000)

### Llama el servicio de Datos.gov.co para consultar los datos con formato .json

In [5]:
results_df = pd.DataFrame.from_records(results)
results_df.describe()

,id_de_caso,fecha_de_notificaci_n,c_digo_divipola,ciudad_de_ubicaci_n,departamento,atenci_n,edad,sexo,tipo,estado,pa_s_de_procedencia,fis,fecha_diagnostico,fecha_recuperado,fecha_reporte_web,tipo_recuperaci_n,codigo_departamento,codigo_pais,pertenencia_etnica,nombre_grupo_etnico,fecha_de_muerte
count,658456,658456,658456,658456,658456,658456,658456,658456,658456,658456,965,658456,654476,507775,658456,507770,658456,961,283455,11325,22507
unique,658456,186,1029,955,37,6,110,4,7,6,50,631,182,175,181,2,33,43,4,1289,169
top,585819,2020-07-27T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Recuperado,30,M,En estudio,Leve,ESPAÑA,Asintomático,2020-08-11T00:00:00.000,2020-08-23T00:00:00.000,2020-08-19T00:00:00.000,Tiempo,11,724,Otro,Sin Comunidad,2020-07-30T00:00:00.000
freq,1,12111,223522,223522,223522,507770,17588,337365,625842,533699,265,79542,17466,14151,13056,411168,223522,265,252661,2829,374


In [6]:
results_df.tail(5)

,id_de_caso,fecha_de_notificaci_n,c_digo_divipola,ciudad_de_ubicaci_n,departamento,atenci_n,edad,sexo,tipo,estado,pa_s_de_procedencia,fis,fecha_diagnostico,fecha_recuperado,fecha_reporte_web,tipo_recuperaci_n,codigo_departamento,codigo_pais,pertenencia_etnica,nombre_grupo_etnico,fecha_de_muerte
658451,658492,2020-08-22T00:00:00.000,05001,Medellín,Antioquia,Casa,18,M,En estudio,Leve,NaN,2020-08-18T00:00:00.000,2020-09-02T00:00:00.000,NaN,2020-09-05T00:00:00.000,NaN,5,NaN,NaN,NaN,NaN
658452,658493,2020-08-22T00:00:00.000,66001,Pereira,Risaralda,Casa,48,F,En estudio,Leve,NaN,2020-08-19T00:00:00.000,2020-09-02T00:00:00.000,NaN,2020-09-05T00:00:00.000,NaN,66,NaN,NaN,NaN,NaN
658453,658494,2020-08-23T00:00:00.000,05001,Medellín,Antioquia,Casa,31,F,En estudio,Leve,NaN,2020-08-19T00:00:00.000,2020-09-03T00:00:00.000,NaN,2020-09-05T00:00:00.000,NaN,5,NaN,NaN,NaN,NaN
658454,658495,2020-08-24T00:00:00.000,25754,Soacha,Cundinamarca,Casa,18,F,En estudio,Leve,NaN,2020-08-20T00:00:00.000,2020-09-04T00:00:00.000,NaN,2020-09-05T00:00:00.000,NaN,25,NaN,NaN,NaN,NaN
658455,658496,2020-08-24T00:00:00.000,73001,Ibagué,Tolima,Casa,24,M,En estudio,Leve,NaN,2020-08-21T00:00:00.000,2020-09-04T00:00:00.000,NaN,2020-09-05T00:00:00.000,NaN,73,NaN,NaN,NaN,NaN


# Análisis exploratorio de los datos


In [7]:
##Exploración inicial de las columnas
results_df.count()

id_de_caso               658456
fecha_de_notificaci_n    658456
c_digo_divipola          658456
ciudad_de_ubicaci_n      658456
departamento             658456
atenci_n                 658456
edad                     658456
sexo                     658456
tipo                     658456
estado                   658456
pa_s_de_procedencia         965
fis                      658456
fecha_diagnostico        654476
fecha_recuperado         507775
fecha_reporte_web        658456
tipo_recuperaci_n        507770
codigo_departamento      658456
codigo_pais                 961
pertenencia_etnica       283455
nombre_grupo_etnico       11325
fecha_de_muerte           22507
dtype: int64

In [8]:
results_df.shape

(658456, 21)

In [9]:
##Revisión de datos nulos en las para identificar nulos en columnas de interés
results_df.isnull().sum()

id_de_caso                    0
fecha_de_notificaci_n         0
c_digo_divipola               0
ciudad_de_ubicaci_n           0
departamento                  0
atenci_n                      0
edad                          0
sexo                          0
tipo                          0
estado                        0
pa_s_de_procedencia      657491
fis                           0
fecha_diagnostico          3980
fecha_recuperado         150681
fecha_reporte_web             0
tipo_recuperaci_n        150686
codigo_departamento           0
codigo_pais              657495
pertenencia_etnica       375001
nombre_grupo_etnico      647131
fecha_de_muerte          635949
dtype: int64

In [10]:
results_df.head()

,id_de_caso,fecha_de_notificaci_n,c_digo_divipola,ciudad_de_ubicaci_n,departamento,atenci_n,edad,sexo,tipo,estado,pa_s_de_procedencia,fis,fecha_diagnostico,fecha_recuperado,fecha_reporte_web,tipo_recuperaci_n,codigo_departamento,codigo_pais,pertenencia_etnica,nombre_grupo_etnico,fecha_de_muerte
0,1,2020-03-02T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Recuperado,19,F,Importado,Leve,ITALIA,2020-02-27T00:00:00.000,2020-03-06T00:00:00.000,2020-03-13T00:00:00.000,2020-03-06T00:00:00.000,PCR,11,380,Otro,NaN,NaN
1,2,2020-03-06T00:00:00.000,76111,Guadalajara de Buga,Valle del Cauca,Recuperado,34,M,Importado,Leve,ESPAÑA,2020-03-04T00:00:00.000,2020-03-09T00:00:00.000,2020-03-19T00:00:00.000,2020-03-09T00:00:00.000,PCR,76,724,Otro,NaN,NaN
2,3,2020-03-07T00:00:00.000,05001,Medellín,Antioquia,Recuperado,50,F,Importado,Leve,ESPAÑA,2020-02-29T00:00:00.000,2020-03-09T00:00:00.000,2020-03-15T00:00:00.000,2020-03-09T00:00:00.000,PCR,5,724,Otro,NaN,NaN
3,4,2020-03-09T00:00:00.000,05001,Medellín,Antioquia,Recuperado,55,M,Relacionado,Leve,NaN,2020-03-06T00:00:00.000,2020-03-11T00:00:00.000,2020-03-26T00:00:00.000,2020-03-11T00:00:00.000,PCR,5,NaN,Otro,NaN,NaN
4,5,2020-03-09T00:00:00.000,05001,Medellín,Antioquia,Recuperado,25,M,Relacionado,Leve,NaN,2020-03-08T00:00:00.000,2020-03-11T00:00:00.000,2020-03-23T00:00:00.000,2020-03-11T00:00:00.000,PCR,5,NaN,Otro,NaN,NaN


In [11]:
##Identificación de los tipos de datos
results_df.dtypes

id_de_caso               object
fecha_de_notificaci_n    object
c_digo_divipola          object
ciudad_de_ubicaci_n      object
departamento             object
atenci_n                 object
edad                     object
sexo                     object
tipo                     object
estado                   object
pa_s_de_procedencia      object
fis                      object
fecha_diagnostico        object
fecha_recuperado         object
fecha_reporte_web        object
tipo_recuperaci_n        object
codigo_departamento      object
codigo_pais              object
pertenencia_etnica       object
nombre_grupo_etnico      object
fecha_de_muerte          object
dtype: object

#Limpieza, transformación y filtrado del conjunto de datos 

In [12]:
##Renombrar columnas
df = (results_df[results_df["estado"] != "N/A"])[["id_de_caso", "fecha_de_notificaci_n", "ciudad_de_ubicaci_n", "atenci_n", "edad", "sexo", "tipo", "estado", "fis", "fecha_diagnostico", "fecha_recuperado", "fecha_reporte_web", "tipo_recuperaci_n", "fecha_de_muerte"]]

In [13]:
df.columns = ["ID_de_caso", "Fecha_de_notificación", "Ciudad_de_ubicación", "atención", "Edad", "Sexo", "Tipo", "Estado", "FIS", "Fecha_diagnostico", "Fecha_recuperado", "fecha_reporte_web", "Tipo_recuperación", "Fecha_de_muerte"]

In [14]:
##Selección de las ciudades principales del conjunto de datos inicial
df = df[df.Ciudad_de_ubicación.isin(["Bogotá D.C.", "Medellín", "Barranquilla", "Cali", "Cartagena de Indias"])]

In [15]:
df.shape

(370068, 14)

In [16]:
df.isnull().sum()

ID_de_caso                    0
Fecha_de_notificación         0
Ciudad_de_ubicación           0
atención                      0
Edad                          0
Sexo                          0
Tipo                          0
Estado                        0
FIS                           0
Fecha_diagnostico          2126
Fecha_recuperado          80072
fecha_reporte_web             0
Tipo_recuperación         80072
Fecha_de_muerte          359693
dtype: int64

### Cambio del formato de la fecha

In [17]:
##Se selecciona de todo el conjunto de datos las columnas de interés: Fecha, atención y ciudad de unicación
new_df = df[["Fecha_diagnostico", "atención", "Ciudad_de_ubicación"]]

In [18]:
new_df['Fecha_diagnostico'] = pd.to_datetime(new_df['Fecha_diagnostico'])
new_df.head(5)

,Fecha_diagnostico,atención,Ciudad_de_ubicación
0,2020-03-06,Recuperado,Bogotá D.C.
2,2020-03-09,Recuperado,Medellín
3,2020-03-11,Recuperado,Medellín
4,2020-03-11,Recuperado,Medellín
6,2020-03-11,Recuperado,Cartagena de Indias


### Dataframe para procesamiento

In [19]:
##Se usa la columna atención para crear la clasificación de los casos. Los casos que se encuentra en NA se seguirán considerando como casos activos mientras no sean reportados como recuperados o muertes
new_df["atención"] = new_df["atención"].replace(to_replace=["Casa", "Hospital", "Hospital UCI", "N/A"], value="ACTIVOS")
new_df["atención"] = new_df["atención"].replace(to_replace=["Recuperado"], value="RECUPERADOS")
new_df["atención"] = new_df["atención"].replace(to_replace=["Fallecido"], value="MUERTES")

In [20]:
##Se creaan las columnas para procesamiento partiendo de los siguientes supuestos: 
##1. Todos los casos reportados son casos confirmados,
##2. Si un caso está reportado como recuperado no se puede clasificar como muerte o como Activo,
##3. Un caso reportado como activo no puede ser clasificado como muerto o recuperado
##4. Un caso activo no puede ser clasificado como muerto o recuperado

new_df["Confirmados"]=1
new_df["Recuperados"]= np.where(new_df["atención"] == "RECUPERADOS", 1, 0)
new_df["Muertos"]= np.where(new_df["atención"] == "MUERTES", 1, 0)
new_df["Activos"]= np.where(new_df["atención"] == "ACTIVOS", 1, 0)
new_df.head()

,Fecha_diagnostico,atención,Ciudad_de_ubicación,Confirmados,Recuperados,Muertos,Activos
0,2020-03-06,RECUPERADOS,Bogotá D.C.,1,1,0,0
2,2020-03-09,RECUPERADOS,Medellín,1,1,0,0
3,2020-03-11,RECUPERADOS,Medellín,1,1,0,0
4,2020-03-11,RECUPERADOS,Medellín,1,1,0,0
6,2020-03-11,RECUPERADOS,Cartagena de Indias,1,1,0,0


In [21]:
new_df["Fecha_diagnostico"]=pd.to_datetime(new_df["Fecha_diagnostico"])

#Ciudad de Barranquilla

In [22]:
##Se filtran los datos para la ciudad de barranquilla
df_agrupado_Barr = new_df[new_df['Ciudad_de_ubicación'] == 'Barranquilla']
df_agrupado_Barr.head()

,Fecha_diagnostico,atención,Ciudad_de_ubicación,Confirmados,Recuperados,Muertos,Activos
58,2020-03-16,RECUPERADOS,Barranquilla,1,1,0,0
60,2020-03-16,RECUPERADOS,Barranquilla,1,1,0,0
103,2020-03-19,RECUPERADOS,Barranquilla,1,1,0,0
125,2020-03-19,RECUPERADOS,Barranquilla,1,1,0,0
126,2020-03-19,RECUPERADOS,Barranquilla,1,1,0,0


In [23]:
df_agrupado2_Barr= df_agrupado_Barr.groupby(["Fecha_diagnostico"]).agg({"Confirmados":'sum',"Recuperados":'sum',"Activos":'sum',"Muertos":'sum'})
df_agrupado2_Barr.head()

,Confirmados,Recuperados,Activos,Muertos
Fecha_diagnostico,,,,
2020-03-16,2,2,0,0
2020-03-19,3,3,0,0
2020-03-21,2,2,0,0
2020-03-24,2,2,0,0
2020-03-25,3,3,0,0


In [24]:
##Se crean las columnas con la frecuencia acumulada por tipo de casos

df_agrupado2_Barr["Confirmados_acum"] =df_agrupado2_Barr["Confirmados"].cumsum()
df_agrupado2_Barr["Recuperados_acum"] =df_agrupado2_Barr["Recuperados"].cumsum()
df_agrupado2_Barr["Activos_acum"] =df_agrupado2_Barr["Activos"].cumsum()
df_agrupado2_Barr["Muertos_acum"] =df_agrupado2_Barr["Muertos"].cumsum()
df_agrupado2_Barr.head()

,Confirmados,Recuperados,Activos,Muertos,Confirmados_acum,Recuperados_acum,Activos_acum,Muertos_acum
Fecha_diagnostico,,,,,,,,
2020-03-16,2,2,0,0,2,2,0,0
2020-03-19,3,3,0,0,5,5,0,0
2020-03-21,2,2,0,0,7,7,0,0
2020-03-24,2,2,0,0,9,9,0,0
2020-03-25,3,3,0,0,12,12,0,0


In [25]:
df_agrupado2_Barr["Días"]=df_agrupado2_Barr.index-df_agrupado2_Barr.index.min()
df_agrupado2_Barr["Días"]=df_agrupado2_Barr["Días"].dt.days

In [26]:
df_agrupado2_Barr.tail(10)

,Confirmados,Recuperados,Activos,Muertos,Confirmados_acum,Recuperados_acum,Activos_acum,Muertos_acum,Días
Fecha_diagnostico,,,,,,,,,
2020-08-27,82,3,79,0,34664,32539,573,1552,164
2020-08-28,57,2,54,1,34721,32541,627,1553,165
2020-08-29,58,1,57,0,34779,32542,684,1553,166
2020-08-30,63,1,62,0,34842,32543,746,1553,167
2020-08-31,70,0,70,0,34912,32543,816,1553,168
2020-09-01,55,1,54,0,34967,32544,870,1553,169
2020-09-02,221,0,221,0,35188,32544,1091,1553,170
2020-09-03,57,0,57,0,35245,32544,1148,1553,171
2020-09-04,81,2,79,0,35326,32546,1227,1553,172


In [27]:
##Se eliminan los últimos 4 días debido a que se observan retrasos en la información de recuperados y muertes
df_agrupado2_Barr = df_agrupado2_Barr[:-4]


In [28]:
df_agrupado2_Barr.tail(10)

,Confirmados,Recuperados,Activos,Muertos,Confirmados_acum,Recuperados_acum,Activos_acum,Muertos_acum,Días
Fecha_diagnostico,,,,,,,,,
2020-08-23,58,29,29,0,34278,32443,290,1545,160
2020-08-24,81,53,26,2,34359,32496,316,1547,161
2020-08-25,154,37,115,2,34513,32533,431,1549,162
2020-08-26,69,3,63,3,34582,32536,494,1552,163
2020-08-27,82,3,79,0,34664,32539,573,1552,164
2020-08-28,57,2,54,1,34721,32541,627,1553,165
2020-08-29,58,1,57,0,34779,32542,684,1553,166
2020-08-30,63,1,62,0,34842,32543,746,1553,167
2020-08-31,70,0,70,0,34912,32543,816,1553,168


#Resumen inicial estado actual Barranquilla

In [29]:
##Se presenta un resumen general acumulado del estado actual de casos

print("Informacion General Barranquilla")
print("Número total de casos confirmados en Barranquilla: ",df_agrupado2_Barr["Confirmados_acum"].iloc[-1])
print("Número total de casos recuperados en Barranquilla: ",df_agrupado2_Barr["Recuperados_acum"].iloc[-1])
print("Número total de muertes en Barranquilla: ",df_agrupado2_Barr["Muertos_acum"].iloc[-1])
print("Número total de casos Activos en Barranquilla: ",df_agrupado2_Barr["Activos_acum"].iloc[-1])
print("Número aproximado de casos confirmados por día: ",np.round(df_agrupado2_Barr["Confirmados_acum"].iloc[-1]/df_agrupado2_Barr.shape[0]))
print("Número aproximado de casos recuperados por día: ",np.round(df_agrupado2_Barr["Recuperados_acum"].iloc[-1]/df_agrupado2_Barr.shape[0]))
print("Número aproximado de muertes por día: ",np.round(df_agrupado2_Barr["Muertos_acum"].iloc[-1]/df_agrupado2_Barr.shape[0]))
print("Nuevos casos confirmados: ",df_agrupado2_Barr["Confirmados_acum"].iloc[-1]-df_agrupado2_Barr["Confirmados_acum"].iloc[-2])

Informacion General Barranquilla
Número total de casos confirmados en Barranquilla:  34967
Número total de casos recuperados en Barranquilla:  32544
Número total de muertes en Barranquilla:  1553
Número total de casos Activos en Barranquilla:  870
Número aproximado de casos confirmados por día:  216.0
Número aproximado de casos recuperados por día:  201.0
Número aproximado de muertes por día:  10.0
Nuevos casos confirmados:  55


In [30]:
##Se grafica el comportamiento del crecimiento semanal por tipo de casos

df_agrupado2_Barr["Semana"]=df_agrupado2_Barr.index.weekofyear

semana_num=[]
semana_confirmados=[]
semana_recuperados=[]
semana_muertos=[]
w=1
for i in list(df_agrupado2_Barr["Semana"].unique()):
    semana_confirmados.append(df_agrupado2_Barr[df_agrupado2_Barr["Semana"]==i]["Confirmados_acum"].iloc[-1])
    semana_recuperados.append(df_agrupado2_Barr[df_agrupado2_Barr["Semana"]==i]["Recuperados_acum"].iloc[-1])
    semana_muertos.append(df_agrupado2_Barr[df_agrupado2_Barr["Semana"]==i]["Muertos_acum"].iloc[-1])
    semana_num.append(w)
    w=w+1

crecimiento=go.Figure()
crecimiento.add_trace(go.Scatter(x=semana_num, y=semana_confirmados,
                    mode='lines+markers',
                    name='Crecimiento semanal casos confirmados'))
crecimiento.add_trace(go.Scatter(x=semana_num, y=semana_recuperados,
                    mode='lines+markers',
                    name='Crecimiento semanal casos recuperados'))
crecimiento.add_trace(go.Scatter(x=semana_num, y=semana_muertos,
                    mode='lines+markers',
                    name='Crecimiento semanal muertos'))
crecimiento.update_layout(title="Crecimiento semanal por tipo de casos en Barranquilla",
                 xaxis_title="Semana",yaxis_title="Número de casos",legend=dict(x=0,y=1,traceorder="normal"))
crecimiento.show()

In [31]:
##Se calcula las tasas de mortalidad y recuperación

df_agrupado2_Barr["Tasa de mortalidad"]=(df_agrupado2_Barr["Muertos_acum"]/df_agrupado2_Barr["Confirmados_acum"])*100
df_agrupado2_Barr["Tasa de recuperación"]=(df_agrupado2_Barr["Recuperados_acum"]/df_agrupado2_Barr["Confirmados_acum"])*100
df_agrupado2_Barr["Tasa de crecimiento"]=(df_agrupado2_Barr["Confirmados"]/df_agrupado2_Barr["Confirmados_acum"])*100
                    

print("Tasa de mortalidad promedio",df_agrupado2_Barr["Tasa de mortalidad"].mean())
print("Tasa de recuperación promedio",df_agrupado2_Barr["Tasa de recuperación"].mean())
print("Tasa de crecimiento promedio",df_agrupado2_Barr["Tasa de crecimiento"].mean())


Tasa de mortalidad promedio 5.212361461127023
Tasa de recuperación promedio 94.56108280801406
Tasa de crecimiento promedio 6.1404891064853615


In [32]:
print("Se presenta la predicción de muertes en la ciudad de Barranquilla, cuya tasa de mortalidad promedio es:",df_agrupado2_Barr["Tasa de mortalidad"].mean())
print("Se presenta la predicción de casos recuperados en la ciudad de Barranquilla, cuya tasa de recuperación promedio es:",df_agrupado2_Barr["Tasa de recuperación"].mean())
print("Se presenta el crecimiento de los casos confirmados en la ciudad de Barranquilla, cuya tasa de crecimiento promedio es:",df_agrupado2_Barr["Tasa de crecimiento"].mean())


Se presenta la predicción de muertes en la ciudad de Barranquilla, cuya tasa de mortalidad promedio es: 5.212361461127023
Se presenta la predicción de casos recuperados en la ciudad de Barranquilla, cuya tasa de recuperación promedio es: 94.56108280801406
Se presenta el crecimiento de los casos confirmados en la ciudad de Barranquilla, cuya tasa de crecimiento promedio es: 6.1404891064853615


In [33]:
##Se grafica el comportamiento de las tasas

Tasas = make_subplots(rows=2, cols=1,
                   subplot_titles=("Tasa de recuperación", "Tasa de mortalidad"))
Tasas.add_trace(
    go.Scatter(x=df_agrupado2_Barr.index, y=(df_agrupado2_Barr["Recuperados_acum"]/df_agrupado2_Barr["Confirmados_acum"])*100,name="Tasa de recuperación"),
    row=1, col=1
)
Tasas.add_trace(
    go.Scatter(x=df_agrupado2_Barr.index, y=(df_agrupado2_Barr["Muertos_acum"]/df_agrupado2_Barr["Confirmados_acum"])*100,name="Tasa de mortalidad"),
    row=2, col=1
)
Tasas.update_layout(height=1000,legend=dict(x=-0.1,y=1.2,traceorder="normal"))
Tasas.update_xaxes(title_text="Fecha", row=1, col=1)
Tasas.update_yaxes(title_text="tasa de recuperación", row=1, col=1)
Tasas.update_xaxes(title_text="Fecha", row=1, col=2)
Tasas.update_yaxes(title_text="Tasa de mortalidad", row=1, col=2)
Tasas.show()

In [34]:
##Se grafica el factor de crecimiento de los tipos de casos
Factor_Crecimiento=go.Figure()
Factor_Crecimiento.add_trace(go.Scatter(x=df_agrupado2_Barr.index, y=df_agrupado2_Barr["Confirmados"]/df_agrupado2_Barr["Confirmados"].shift(),
                    mode='lines',
                    name='Factor de crecimiento casos confirmados'))
Factor_Crecimiento.add_trace(go.Scatter(x=df_agrupado2_Barr.index, y=df_agrupado2_Barr["Recuperados"]/df_agrupado2_Barr["Recuperados"].shift(),
                    mode='lines',
                    name='Factor de crecimiento casos recuperados'))
Factor_Crecimiento.add_trace(go.Scatter(x=df_agrupado2_Barr.index, y=df_agrupado2_Barr["Muertos"]/df_agrupado2_Barr["Muertos"].shift(),
                    mode='lines',
                    name='Factor de crecimiento muertes'))
Factor_Crecimiento.update_layout(title="Factor de crecimiento casos confirmados, recuperados y muertes",
                 xaxis_title="Fecha",yaxis_title="Factor de crecimiento",
                 legend=dict(x=0,y=-0.4,traceorder="normal"))
Factor_Crecimiento.show()

#Modelo de predicción AR para Barranquilla


##Casos confirmados

In [35]:
##Entrenamiento 
model_train_Barr_Con=df_agrupado2_Barr.iloc[:int(df_agrupado2_Barr.shape[0]*0.95)]
valid_Barr_Con=df_agrupado2_Barr.iloc[int(df_agrupado2_Barr.shape[0]*0.95):]
y_pred_Barr_Con=valid_Barr_Con.copy()

In [36]:
##Selección del modelo
model_ar_Barr_Con= auto_arima(model_train_Barr_Con["Confirmados_acum"],trace=True, error_action='ignore', start_p=0,start_q=0,max_p=5,max_q=0,
                   suppress_warnings=True,stepwise=False,seasonal=False)
model_ar_Barr_Con.fit(model_train_Barr_Con["Confirmados_acum"])

Fit ARIMA: order=(0, 2, 0); AIC=1915.670, BIC=1921.705, Fit time=0.022 seconds
Fit ARIMA: order=(1, 2, 0); AIC=1860.697, BIC=1869.749, Fit time=0.079 seconds
Fit ARIMA: order=(2, 2, 0); AIC=1855.884, BIC=1867.953, Fit time=0.058 seconds
Fit ARIMA: order=(3, 2, 0); AIC=1857.076, BIC=1872.162, Fit time=0.132 seconds
Fit ARIMA: order=(4, 2, 0); AIC=1857.651, BIC=1875.755, Fit time=0.124 seconds
Fit ARIMA: order=(5, 2, 0); AIC=1854.275, BIC=1875.396, Fit time=0.211 seconds
Total fit time: 0.667 seconds


ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(5, 2, 0),
      out_of_sample_size=0, scoring='mse', scoring_args={}, seasonal_order=None,
      solver='lbfgs', start_params=None, suppress_warnings=True,
      transparams=True, trend='c')

In [37]:
##Calculo predicción
prediction_ar_Barr_Con=model_ar_Barr_Con.predict(len(valid_Barr_Con))
y_pred_Barr_Con["AR Predicción"]=prediction_ar_Barr_Con

In [38]:
##Métrica RMSE
model_scores_Barr_Conf=[]
model_scores_Barr_Conf.append(np.sqrt(mean_squared_error(y_pred_Barr_Con["Confirmados_acum"],y_pred_Barr_Con["AR Predicción"])))
print("RMSE para modelo AR: ",np.sqrt(mean_squared_error(y_pred_Barr_Con["Confirmados_acum"],y_pred_Barr_Con["AR Predicción"])))

RMSE para modelo AR:  105.90551916222242


In [39]:
##Graficando el modelo para los confirmados
Confirmados_Barr=go.Figure()
Confirmados_Barr.add_trace(go.Scatter(x=model_train_Barr_Con.index, y=model_train_Barr_Con["Confirmados_acum"],
                    mode='lines+markers',name="Datos de entrenamiento para casos confirmados"))
Confirmados_Barr.add_trace(go.Scatter(x=valid_Barr_Con.index, y=valid_Barr_Con["Confirmados_acum"],
                    mode='lines+markers',name="Datos de Validación para casos confirmados",))
Confirmados_Barr.add_trace(go.Scatter(x=valid_Barr_Con.index, y=y_pred_Barr_Con["AR Predicción"],
                    mode='lines+markers',name="Predicción de casos confirmados",))
Confirmados_Barr.update_layout(title="Predicción casos confirmados modelo AR",
                 xaxis_title="Fecha",yaxis_title="Casos  confirmados",legend=dict(x=0,y=1,traceorder="normal"))
Confirmados_Barr.show()

In [40]:
#Crea datos de predicción a 30 días
nuevo_dia=[]
Prediccion_AR_Barr_Con=[]
for i in range(1,30):
    nuevo_dia.append(df_agrupado2_Barr.index[-1]+timedelta(days=i))
    Prediccion_AR_Barr_Con.append(model_ar_Barr_Con.predict(len(valid_Barr_Con)+i)[-1])
    

##Casos recuperados

In [41]:
##Se debe considerar que se observan retrasos en el reporte de casos al mostrar de forma consecutiva 0 recuperados y 0 muertes, teniendo en cuenta que aún hay casos activos en la región y que los casos no permanecen activos indefinidamente 
##Entrenamiento
model_train_Barr_recu=df_agrupado2_Barr.iloc[:int(df_agrupado2_Barr.shape[0]*0.95)]
valid_Barr_recu=df_agrupado2_Barr.iloc[int(df_agrupado2_Barr.shape[0]*0.95):]
y_pred_Barr_recu=valid_Barr_recu.copy()

In [42]:
##Selección
model_ar_Barr_recu= auto_arima(model_train_Barr_recu["Recuperados"],trace=True, error_action='ignore', start_p=0,start_q=0,max_p=7,max_q=0,
                   suppress_warnings=True,stepwise=False,seasonal=False)
model_ar_Barr_recu.fit(model_train_Barr_recu["Recuperados"])

Fit ARIMA: order=(0, 1, 0); AIC=1918.064, BIC=1924.112, Fit time=0.005 seconds
Fit ARIMA: order=(1, 1, 0); AIC=1862.359, BIC=1871.431, Fit time=0.053 seconds
Fit ARIMA: order=(2, 1, 0); AIC=1857.066, BIC=1869.162, Fit time=0.059 seconds
Fit ARIMA: order=(3, 1, 0); AIC=1858.214, BIC=1873.333, Fit time=0.083 seconds
Fit ARIMA: order=(4, 1, 0); AIC=1859.118, BIC=1877.261, Fit time=0.141 seconds
Fit ARIMA: order=(5, 1, 0); AIC=1856.172, BIC=1877.340, Fit time=0.247 seconds
Fit ARIMA: order=(6, 1, 0); AIC=1858.039, BIC=1882.230, Fit time=0.390 seconds
Fit ARIMA: order=(7, 1, 0); AIC=1859.670, BIC=1886.885, Fit time=0.766 seconds
Total fit time: 1.754 seconds


ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(5, 1, 0),
      out_of_sample_size=0, scoring='mse', scoring_args={}, seasonal_order=None,
      solver='lbfgs', start_params=None, suppress_warnings=True,
      transparams=True, trend='c')

In [43]:
##Predicción
prediction_ar_Barr_recu=model_ar_Barr_recu.predict(len(valid_Barr_recu))
y_pred_Barr_recu["AR Predicción"]=prediction_ar_Barr_recu

In [44]:
#Métrica RMSE
model_scores_Barr_recu=[]
model_scores_Barr_recu.append(np.sqrt(mean_squared_error(y_pred_Barr_recu["Recuperados"],y_pred_Barr_recu["AR Predicción"])))
print("RMSE para modelo AR: ",np.sqrt(mean_squared_error(y_pred_Barr_recu["Recuperados"],y_pred_Barr_recu["AR Predicción"])))

RMSE para modelo AR:  53.171893060443956


In [45]:
##Graficando el modelo para los recuperados
Recuperados_Barr=go.Figure()
Recuperados_Barr.add_trace(go.Scatter(x=model_train_Barr_recu.index, y=model_train_Barr_recu["Recuperados"],
                    mode='lines+markers',name="Datos de entrenamiento para casos recuperados"))
Recuperados_Barr.add_trace(go.Scatter(x=valid_Barr_recu.index, y=valid_Barr_recu["Recuperados"],
                    mode='lines+markers',name="Datos de Validación para casos recuperados",))
Recuperados_Barr.add_trace(go.Scatter(x=valid_Barr_recu.index, y=y_pred_Barr_recu["AR Predicción"],
                    mode='lines+markers',name="Predicción de casos recuperados",))
Recuperados_Barr.update_layout(title="Predicción casos recuperados modelo AR",
                 xaxis_title="Fecha",yaxis_title="Casos  recuperados",legend=dict(x=0,y=1,traceorder="normal"))
Recuperados_Barr.show()


In [46]:
#Crea datos de predicción a 30 días
Prediccion_AR_Barr_recu=[]
for i in range(1,30):
    Prediccion_AR_Barr_recu.append(model_ar_Barr_recu.predict(len(valid_Barr_recu)+i)[-1])


##Muertes

In [47]:
##Entrenamiento
model_train_Barr_M=df_agrupado2_Barr.iloc[:int(df_agrupado2_Barr.shape[0]*0.90)]
valid_Barr_M=df_agrupado2_Barr.iloc[int(df_agrupado2_Barr.shape[0]*0.90):]
y_pred_Barr_M=valid_Barr_M.copy()

In [48]:
##Selección
model_sarima_Barr_M= auto_arima(model_train_Barr_M["Muertos"],trace=True, error_action='ignore', 
                         start_p=0,start_q=0,max_p=2,max_q=2,m=9,
                   suppress_warnings=True,stepwise=True,seasonal=True)
model_sarima_Barr_M.fit(model_train_Barr_M["Muertos"])


Fit ARIMA: order=(0, 1, 0) seasonal_order=(1, 0, 1, 9); AIC=1047.294, BIC=1059.173, Fit time=0.256 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 0, 0, 9); AIC=1044.414, BIC=1050.353, Fit time=0.026 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 0, 0, 9); AIC=1005.253, BIC=1017.133, Fit time=0.146 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 0, 1, 9); AIC=995.767, BIC=1007.646, Fit time=0.146 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 0, 1, 9); AIC=996.335, BIC=1011.184, Fit time=0.252 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 0, 0, 9); AIC=995.748, BIC=1004.658, Fit time=0.046 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 0, 0, 9); AIC=997.748, BIC=1009.627, Fit time=0.083 seconds
Fit ARIMA: order=(0, 1, 2) seasonal_order=(0, 0, 0, 9); AIC=997.748, BIC=1009.627, Fit time=0.091 seconds
Fit ARIMA: order=(1, 1, 2) seasonal_order=(0, 0, 0, 9); AIC=999.470, BIC=1014.319, Fit time=0.163 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(

ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(0, 1, 1),
      out_of_sample_size=0, scoring='mse', scoring_args={},
      seasonal_order=(1, 0, 0, 9), solver='lbfgs', start_params=None,
      suppress_warnings=True, transparams=True, trend='c')

In [49]:
#Predicción
prediction_sarima_Barr_M=model_sarima_Barr_M.predict(len(valid_Barr_M))
y_pred_Barr_M["SARIMA Model Prediction"]=prediction_sarima_Barr_M

In [50]:
#Métricas RMSE
model_scores_Barr_M=[]
model_scores_Barr_M.append(np.sqrt(mean_squared_error(y_pred_Barr_M["Muertos"],y_pred_Barr_M["SARIMA Model Prediction"])))
print("RMSE para modelo SARIMA: ",np.sqrt(mean_squared_error(y_pred_Barr_M["Muertos"],y_pred_Barr_M["SARIMA Model Prediction"])))

RMSE para modelo SARIMA:  1.6767109360340717


In [51]:
#Gráfica SARIMA Muertes
Muertes_Barr=go.Figure()
Muertes_Barr.add_trace(go.Scatter(x=model_train_Barr_M.index, y=model_train_Barr_M["Muertos"],
                    mode='lines+markers',name="Datos de entrenamiento para muertes"))
Muertes_Barr.add_trace(go.Scatter(x=valid_Barr_M.index, y=valid_Barr_M["Muertos"],
                    mode='lines+markers',name="Datos de validación para muertes",))
Muertes_Barr.add_trace(go.Scatter(x=valid_Barr_M.index, y=y_pred_Barr_M["SARIMA Model Prediction"],
                    mode='lines+markers',name="Predicción para muertes",))
Muertes_Barr.update_layout(title="Predicción muertes modelo SARIMA",
                 xaxis_title="Fecha",yaxis_title="Muertes",legend=dict(x=0,y=1,traceorder="normal"))
Muertes_Barr.show()

In [52]:
#Se crean datos de predicción a 30 días
Prediccion_SARIMA_Barr_M=[]
for i in range(1,30):
    Prediccion_SARIMA_Barr_M.append(model_sarima_Barr_M.predict(len(valid_Barr_M)+i)[-1])

#Resumen general predicción casos confirmados, nuevos, activos, recuperados y muertes Barranquilla

In [53]:
##Se tiene en cuenta que dada la reapertura se tiene una expectativa de crecimiento de los nuevos casospor lo cual se consideran los modelos evaluados pensando en ese escenario

#pd.set_option('display.float_format', lambda x: '%.6f' % x)
#Predicciones_Barr=pd.DataFrame(zip(nuevo_dia,Prediccion_AR_Barr_Con,Prediccion_AR_Barr_recu,Prediccion_SARIMA_Barr_M ), columns=["Fechas","Modelo AR Confirmados","Modelo AR Recuperados", "Modelo SARIMA Muertes"])
#Predicciones_Barr.head()

pd.set_option('display.float_format', lambda x: '%.6f' % x)

Predicciones_Barr=zip(nuevo_dia,Prediccion_AR_Barr_Con,Prediccion_AR_Barr_recu,Prediccion_SARIMA_Barr_M)

Predicciones_Barr = pd.DataFrame(list(Predicciones_Barr), columns=["Fechas","Modelo AR Confirmados","Modelo AR Recuperados", "Modelo SARIMA Muertes"])

Predicciones_Barr.head()


,Fechas,Modelo AR Confirmados,Modelo AR Recuperados,Modelo SARIMA Muertes
0,2020-09-02,35278.962410,62.993409,2.261674
1,2020-09-03,35382.159463,62.187901,2.286894
2,2020-09-04,35486.047313,62.555102,2.292121
3,2020-09-05,35591.262299,63.429259,2.306043
4,2020-09-06,35696.098344,63.092009,2.326486


In [54]:
##Predicciones 1ra semana
Predicciones_Barr["Activos"]= Predicciones_Barr["Modelo AR Confirmados"]-Predicciones_Barr["Modelo AR Recuperados"]-Predicciones_Barr["Modelo SARIMA Muertes"]
Predicciones_Barr["Nuevos"]= Predicciones_Barr["Modelo AR Confirmados"].diff()

Predicciones_Barr.head(7)

,Fechas,Modelo AR Confirmados,Modelo AR Recuperados,Modelo SARIMA Muertes,Activos,Nuevos
0,2020-09-02,35278.962410,62.993409,2.261674,35213.707326,nan
1,2020-09-03,35382.159463,62.187901,2.286894,35317.684668,103.197054
2,2020-09-04,35486.047313,62.555102,2.292121,35421.200091,103.887850
3,2020-09-05,35591.262299,63.429259,2.306043,35525.526997,105.214986
4,2020-09-06,35696.098344,63.092009,2.326486,35630.679850,104.836045
5,2020-09-07,35801.911499,63.687247,2.327365,35735.896887,105.813155
6,2020-09-08,35908.514180,64.232496,2.345634,35841.936050,106.602681


In [55]:
##Predicciones cuarta semana
Predicciones_Barr.tail(7)

,Fechas,Modelo AR Confirmados,Modelo AR Recuperados,Modelo SARIMA Muertes,Activos,Nuevos
22,2020-09-24,37697.487040,70.056950,2.568949,37624.861141,116.526001
23,2020-09-25,37814.640501,70.428245,2.582682,37741.629573,117.153460
24,2020-09-26,37932.419238,70.797301,2.596707,37859.025230,117.778737
25,2020-09-27,38050.828403,71.170203,2.610622,37977.047577,118.409165
26,2020-09-28,38169.864272,71.540733,2.624647,38095.698892,119.035869
27,2020-09-29,38289.527369,71.911272,2.638562,38214.977535,119.663096
28,2020-09-30,38409.819361,72.283180,2.652538,38334.883643,120.291992


##Intervalos de confianza


In [56]:
Int_Conf = sms.DescrStatsW(y_pred_Barr_Con["AR Predicción"]).tconfint_mean()
Int_Recu = sms.DescrStatsW(y_pred_Barr_recu["AR Predicción"]).tconfint_mean()
Int_Muertos = sms.DescrStatsW(y_pred_Barr_M["SARIMA Model Prediction"]).tconfint_mean()

In [57]:
pd.set_option('display.float_format', lambda x: '%.6f' % x)
Intervalos_Barr=zip(Int_Conf,Int_Recu,Int_Muertos)
Intervalos_Barr=pd.DataFrame(list(Intervalos_Barr), columns=["Int_Confirmados","Int_Recuperados", "Int_Muertes"])
Intervalos_Barr

,Int_Confirmados,Int_Recuperados,Int_Muertes
0,34559.670080,56.726465,2.117683
1,34984.419762,64.047884,2.316819


##Métricas

In [58]:
Error_Barr=zip(model_scores_Barr_Conf,model_scores_Barr_recu,model_scores_Barr_M)
Error_Barr=pd.DataFrame(list(Error_Barr), columns=["RMSE_Confirmados","RMSE_Recuperados", "RMSE_Muertes"])
Error_Barr

,RMSE_Confirmados,RMSE_Recuperados,RMSE_Muertes
0,105.905519,53.171893,1.676711


#Ciudad de Bogotá

In [59]:
##Se filtran los datos para la ciudad de Bogotá
df_agrupado_Bog = new_df[new_df['Ciudad_de_ubicación'] == 'Bogotá D.C.']
df_agrupado_Bog.head()


,Fecha_diagnostico,atención,Ciudad_de_ubicación,Confirmados,Recuperados,Muertos,Activos
0,2020-03-06,RECUPERADOS,Bogotá D.C.,1,1,0,0
7,2020-03-11,RECUPERADOS,Bogotá D.C.,1,1,0,0
8,2020-03-11,RECUPERADOS,Bogotá D.C.,1,1,0,0
9,2020-03-12,RECUPERADOS,Bogotá D.C.,1,1,0,0
10,2020-03-12,RECUPERADOS,Bogotá D.C.,1,1,0,0


In [60]:
df_agrupado2_Bog= df_agrupado_Bog.groupby(["Fecha_diagnostico"]).agg({"Confirmados":'sum',"Recuperados":'sum',"Activos":'sum',"Muertos":'sum'})
df_agrupado2_Bog.head()

,Confirmados,Recuperados,Activos,Muertos
Fecha_diagnostico,,,,
2020-03-06,1,1,0,0
2020-03-11,2,2,0,0
2020-03-12,2,2,0,0
2020-03-13,1,1,0,0
2020-03-14,10,10,0,0


In [61]:
##Se crean las columnas con la frecuencia acumulada por tipo de casos

df_agrupado2_Bog["Confirmados_acum"] =df_agrupado2_Bog["Confirmados"].cumsum()
df_agrupado2_Bog["Recuperados_acum"] =df_agrupado2_Bog["Recuperados"].cumsum()
df_agrupado2_Bog["Activos_acum"] =df_agrupado2_Bog["Activos"].cumsum()
df_agrupado2_Bog["Muertos_acum"] =df_agrupado2_Bog["Muertos"].cumsum()
df_agrupado2_Bog.head()


,Confirmados,Recuperados,Activos,Muertos,Confirmados_acum,Recuperados_acum,Activos_acum,Muertos_acum
Fecha_diagnostico,,,,,,,,
2020-03-06,1,1,0,0,1,1,0,0
2020-03-11,2,2,0,0,3,3,0,0
2020-03-12,2,2,0,0,5,5,0,0
2020-03-13,1,1,0,0,6,6,0,0
2020-03-14,10,10,0,0,16,16,0,0


In [62]:
df_agrupado2_Bog["Días"]=df_agrupado2_Bog.index-df_agrupado2_Bog.index.min()
df_agrupado2_Bog["Días"]=df_agrupado2_Bog["Días"].dt.days

In [63]:
df_agrupado2_Bog.tail(10)

,Confirmados,Recuperados,Activos,Muertos,Confirmados_acum,Recuperados_acum,Activos_acum,Muertos_acum,Días
Fecha_diagnostico,,,,,,,,,
2020-08-27,2220,117,2084,19,205098,164640,34826,5632,174
2020-08-28,2933,117,2793,23,208031,164757,37619,5655,175
2020-08-29,2058,46,1986,26,210089,164803,39605,5681,176
2020-08-30,2029,39,1980,10,212118,164842,41585,5691,177
2020-08-31,3531,72,3437,22,215649,164914,45022,5713,178
2020-09-01,2851,86,2746,19,218500,165000,47768,5732,179
2020-09-02,1821,10,1795,16,220321,165010,49563,5748,180
2020-09-03,2290,0,2281,9,222611,165010,51844,5757,181
2020-09-04,479,0,479,0,223090,165010,52323,5757,182


In [64]:
##Se eliminan los últimos 4 días debido a que se observan retrasos en la información de recuperados y muertes
df_agrupado2_Bog = df_agrupado2_Bog[:-4]

In [65]:
df_agrupado2_Bog.tail(10)

,Confirmados,Recuperados,Activos,Muertos,Confirmados_acum,Recuperados_acum,Activos_acum,Muertos_acum,Días
Fecha_diagnostico,,,,,,,,,
2020-08-23,3476,828,2612,36,193986,163703,24792,5491,170
2020-08-24,3148,341,2754,53,197134,164044,27546,5544,171
2020-08-25,3264,275,2949,40,200398,164319,30495,5584,172
2020-08-26,2480,204,2247,29,202878,164523,32742,5613,173
2020-08-27,2220,117,2084,19,205098,164640,34826,5632,174
2020-08-28,2933,117,2793,23,208031,164757,37619,5655,175
2020-08-29,2058,46,1986,26,210089,164803,39605,5681,176
2020-08-30,2029,39,1980,10,212118,164842,41585,5691,177
2020-08-31,3531,72,3437,22,215649,164914,45022,5713,178


#Resumen inicial estado actual Bogotá

In [66]:
##Se presenta un resumen general acumulado del estado actual de casos

print("Informacion General Bogotá ")
print("Número total de casos confirmados en Bogotá: ",df_agrupado2_Bog["Confirmados_acum"].iloc[-1])
print("Número total de casos recuperados en Bogotá: ",df_agrupado2_Bog["Recuperados_acum"].iloc[-1])
print("Número total de muertes en Bogotá: ",df_agrupado2_Bog["Muertos_acum"].iloc[-1])
print("Número total de casos Activos en Bogotá: ",df_agrupado2_Bog["Activos_acum"].iloc[-1])
print("Número aproximado de casos confirmados por día: ",np.round(df_agrupado2_Bog["Confirmados_acum"].iloc[-1]/df_agrupado2_Bog.shape[0]))
print("Número aproximado de casos recuperados por día: ",np.round(df_agrupado2_Bog["Recuperados_acum"].iloc[-1]/df_agrupado2_Bog.shape[0]))
print("Número aproximado de muertes por día: ",np.round(df_agrupado2_Bog["Muertos_acum"].iloc[-1]/df_agrupado2_Bog.shape[0]))
print("Nuevos casos confirmados: ",df_agrupado2_Bog["Confirmados_acum"].iloc[-1]-df_agrupado2_Bog["Confirmados_acum"].iloc[-2])

Informacion General Bogotá 
Número total de casos confirmados en Bogotá:  218500
Número total de casos recuperados en Bogotá:  165000
Número total de muertes en Bogotá:  5732
Número total de casos Activos en Bogotá:  47768
Número aproximado de casos confirmados por día:  1241.0
Número aproximado de casos recuperados por día:  938.0
Número aproximado de muertes por día:  33.0
Nuevos casos confirmados:  2851


In [67]:
##Se grafica el comportamiento del crecimiento semanal por tipo de casos

df_agrupado2_Bog["Semana"]=df_agrupado2_Bog.index.weekofyear

semana_num=[]
semana_confirmados=[]
semana_recuperados=[]
semana_muertos=[]
w=1
for i in list(df_agrupado2_Bog["Semana"].unique()):
    semana_confirmados.append(df_agrupado2_Bog[df_agrupado2_Bog["Semana"]==i]["Confirmados_acum"].iloc[-1])
    semana_recuperados.append(df_agrupado2_Bog[df_agrupado2_Bog["Semana"]==i]["Recuperados_acum"].iloc[-1])
    semana_muertos.append(df_agrupado2_Bog[df_agrupado2_Bog["Semana"]==i]["Muertos_acum"].iloc[-1])
    semana_num.append(w)
    w=w+1

crecimiento=go.Figure()
crecimiento.add_trace(go.Scatter(x=semana_num, y=semana_confirmados,
                    mode='lines+markers',
                    name='Crecimiento semanal casos confirmados'))
crecimiento.add_trace(go.Scatter(x=semana_num, y=semana_recuperados,
                    mode='lines+markers',
                    name='Crecimiento semanal casos recuperados'))
crecimiento.add_trace(go.Scatter(x=semana_num, y=semana_muertos,
                    mode='lines+markers',
                    name='Crecimiento semanal muertos'))
crecimiento.update_layout(title="Crecimiento semanal por tipo de casos en Bogotá",
                 xaxis_title="Semana",yaxis_title="Número de casos",legend=dict(x=0,y=1,traceorder="normal"))
crecimiento.show()

In [68]:
##Se calcula las tasas de mortalidad y recuperación
df_agrupado2_Bog["Tasa de mortalidad"]=(df_agrupado2_Bog["Muertos_acum"]/df_agrupado2_Bog["Confirmados_acum"])*100
df_agrupado2_Bog["Tasa de recuperación"]=(df_agrupado2_Bog["Recuperados_acum"]/df_agrupado2_Bog["Confirmados_acum"])*100
df_agrupado2_Bog["Tasa de crecimiento"]=(df_agrupado2_Bog["Confirmados"]/df_agrupado2_Bog["Confirmados_acum"])*100
print("Tasa de mortalidad promedio",df_agrupado2_Bog["Tasa de mortalidad"].mean())
print("Tasa de recuperación promedio",df_agrupado2_Bog["Tasa de recuperación"].mean())


Tasa de mortalidad promedio 3.6944501587620264
Tasa de recuperación promedio 94.90838548986535


In [69]:
print("Se presenta la predicción de muertes en la ciudad de Bogotá, cuya tasa de mortalidad promedio es:",df_agrupado2_Bog["Tasa de mortalidad"].mean())
print("Se presenta la predicción de casos recuperados en la ciudad de Bogotá, cuya tasa de recuperación promedio es:",df_agrupado2_Bog["Tasa de recuperación"].mean())
print("Se presenta el crecimiento de los casos confirmados en la ciudad de Bogotá, cuya tasa de crecimiento promedio es:",df_agrupado2_Bog["Tasa de crecimiento"].mean())

Se presenta la predicción de muertes en la ciudad de Bogotá, cuya tasa de mortalidad promedio es: 3.6944501587620264
Se presenta la predicción de casos recuperados en la ciudad de Bogotá, cuya tasa de recuperación promedio es: 94.90838548986535
Se presenta el crecimiento de los casos confirmados en la ciudad de Bogotá, cuya tasa de crecimiento promedio es: 6.7156443601033695


In [70]:
##Se grafica el comportamiento de las tasas
Tasas = make_subplots(rows=2, cols=1,
                   subplot_titles=("Tasa de recuperación", "Tasa de mortalidad"))
Tasas.add_trace(
    go.Scatter(x=df_agrupado2_Bog.index, y=(df_agrupado2_Bog["Recuperados_acum"]/df_agrupado2_Bog["Confirmados_acum"])*100,name="Tasa de recuperación"),
    row=1, col=1
)
Tasas.add_trace(
    go.Scatter(x=df_agrupado2_Bog.index, y=(df_agrupado2_Bog["Muertos_acum"]/df_agrupado2_Bog["Confirmados_acum"])*100,name="Tasa de mortalidad"),
    row=2, col=1
)
Tasas.update_layout(height=1000,legend=dict(x=-0.1,y=1.2,traceorder="normal"))
Tasas.update_xaxes(title_text="Fecha", row=1, col=1)
Tasas.update_yaxes(title_text="tasa de recuperación", row=1, col=1)
Tasas.update_xaxes(title_text="Fecha", row=1, col=2)
Tasas.update_yaxes(title_text="Tasa de mortalidad", row=1, col=2)
Tasas.show()

In [71]:
##Se grafica el factor de crecimiento de los tipos de casos
Factor_Crecimiento=go.Figure()
Factor_Crecimiento.add_trace(go.Scatter(x=df_agrupado2_Bog.index, y=df_agrupado2_Bog["Confirmados"]/df_agrupado2_Bog["Confirmados"].shift(),
                    mode='lines',
                    name='Factor de crecimiento casos confirmados'))
Factor_Crecimiento.add_trace(go.Scatter(x=df_agrupado2_Bog.index, y=df_agrupado2_Bog["Recuperados"]/df_agrupado2_Bog["Recuperados"].shift(),
                    mode='lines',
                    name='Factor de crecimiento casos recuperados'))
Factor_Crecimiento.add_trace(go.Scatter(x=df_agrupado2_Bog.index, y=df_agrupado2_Bog["Muertos"]/df_agrupado2_Bog["Muertos"].shift(),
                    mode='lines',
                    name='Factor de crecimiento muertes'))
Factor_Crecimiento.update_layout(title="Factor de crecimiento casos confirmados, recuperados y muertes",
                 xaxis_title="Fecha",yaxis_title="Factor de crecimiento",
                 legend=dict(x=0,y=-0.4,traceorder="normal"))
Factor_Crecimiento.show()



#Modelo de predicción AR para Bogotá

##Casos confirmados




In [72]:
##Entrenamiento 
model_train_Bog_Con=df_agrupado2_Bog.iloc[:int(df_agrupado2_Bog.shape[0]*0.95)]
valid_Bog_Con=df_agrupado2_Bog.iloc[int(df_agrupado2_Bog.shape[0]*0.95):]
y_pred_Bog_Con=valid_Bog_Con.copy()


In [73]:
##Selección del modelo
model_ar_Bog_Con= auto_arima(model_train_Bog_Con["Confirmados_acum"],trace=True, error_action='ignore', start_p=0,start_q=0,max_p=3,max_q=0,
                   suppress_warnings=True,stepwise=False,seasonal=False)
model_ar_Bog_Con.fit(model_train_Bog_Con["Confirmados_acum"])

Fit ARIMA: order=(0, 2, 0); AIC=2931.949, BIC=2938.161, Fit time=0.003 seconds
Fit ARIMA: order=(1, 2, 0); AIC=2875.896, BIC=2885.214, Fit time=0.027 seconds
Fit ARIMA: order=(2, 2, 0); AIC=2850.498, BIC=2862.922, Fit time=0.051 seconds
Fit ARIMA: order=(3, 2, 0); AIC=2826.118, BIC=2841.648, Fit time=0.196 seconds
Total fit time: 0.285 seconds


ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(3, 2, 0),
      out_of_sample_size=0, scoring='mse', scoring_args={}, seasonal_order=None,
      solver='lbfgs', start_params=None, suppress_warnings=True,
      transparams=True, trend='c')

In [74]:
##Calculo predicción
prediction_ar_Bog_Con=model_ar_Bog_Con.predict(len(valid_Bog_Con))
y_pred_Bog_Con["AR Predicción"]=prediction_ar_Bog_Con

In [75]:
##Métrica RMSE
model_scores_Bog_Conf=[]
model_scores_Bog_Conf.append(np.sqrt(mean_squared_error(y_pred_Bog_Con["Confirmados_acum"],y_pred_Bog_Con["AR Predicción"])))
print("RMSE para modelo AR: ",np.sqrt(mean_squared_error(y_pred_Bog_Con["Confirmados_acum"],y_pred_Bog_Con["AR Predicción"])))

RMSE para modelo AR:  4299.557646650094


In [76]:
##Graficando el modelo para los confirmados
Confirmados_Bog=go.Figure()
Confirmados_Bog.add_trace(go.Scatter(x=model_train_Bog_Con.index, y=model_train_Bog_Con["Confirmados_acum"],
                    mode='lines+markers',name="Datos de entrenamiento para casos confirmados"))
Confirmados_Bog.add_trace(go.Scatter(x=valid_Bog_Con.index, y=valid_Bog_Con["Confirmados_acum"],
                    mode='lines+markers',name="Datos de Validación para casos confirmados",))
Confirmados_Bog.add_trace(go.Scatter(x=valid_Bog_Con.index, y=y_pred_Bog_Con["AR Predicción"],
                    mode='lines+markers',name="Predicción de casos confirmados",))
Confirmados_Bog.update_layout(title="Predicción casos confirmados modelo AR",
                 xaxis_title="Fecha",yaxis_title="Casos  confirmados",legend=dict(x=0,y=1,traceorder="normal"))
Confirmados_Bog.show()

In [77]:
#Crea datos de predicción a 30 días
nuevo_dia=[]
Prediccion_AR_Bog_Con=[]
for i in range(1,30):
    nuevo_dia.append(df_agrupado2_Bog.index[-1]+timedelta(days=i))
    Prediccion_AR_Bog_Con.append(model_ar_Bog_Con.predict(len(valid_Bog_Con)+i)[-1])

##Casos recuperados

In [78]:
##Se debe considerar que se observan retrasos en el reporte de casos al mostrar de forma consecutiva 0 recuperados y 0 muertes, teniendo en cuenta que aún hay casos activos en la región y que los casos no permanecen activos indefinidamente 
##Entrenamiento
model_train_Bog_recu=df_agrupado2_Bog.iloc[:int(df_agrupado2_Bog.shape[0]*0.85)]
valid_Bog_recu=df_agrupado2_Bog.iloc[int(df_agrupado2_Bog.shape[0]*0.85):]
y_pred_Bog_recu=valid_Bog_recu.copy()

In [79]:
##Selección
model_ar_Bog_recu= auto_arima(model_train_Bog_recu["Recuperados"],trace=True, error_action='ignore', start_p=0,start_q=0,max_p=3,max_q=0,
                   suppress_warnings=True,stepwise=False,seasonal=False)
model_ar_Bog_recu.fit(model_train_Bog_recu["Recuperados"])

Fit ARIMA: order=(0, 1, 0); AIC=2462.019, BIC=2468.013, Fit time=0.006 seconds
Fit ARIMA: order=(1, 1, 0); AIC=2449.212, BIC=2458.203, Fit time=0.028 seconds
Fit ARIMA: order=(2, 1, 0); AIC=2428.851, BIC=2440.839, Fit time=0.067 seconds
Fit ARIMA: order=(3, 1, 0); AIC=2411.606, BIC=2426.592, Fit time=0.123 seconds
Total fit time: 0.233 seconds


ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(3, 1, 0),
      out_of_sample_size=0, scoring='mse', scoring_args={}, seasonal_order=None,
      solver='lbfgs', start_params=None, suppress_warnings=True,
      transparams=True, trend='c')

In [80]:
##Predicción
prediction_ar_Bog_recu=model_ar_Bog_recu.predict(len(valid_Bog_recu))
y_pred_Bog_recu["AR Predicción"]=prediction_ar_Bog_recu


In [81]:
#Métrica RMSE
model_scores_Bog_recu=[]
model_scores_Bog_recu.append(np.sqrt(mean_squared_error(y_pred_Bog_recu["Recuperados"],y_pred_Bog_recu["AR Predicción"])))
print("RMSE para modelo AR: ",np.sqrt(mean_squared_error(y_pred_Bog_recu["Recuperados"],y_pred_Bog_recu["AR Predicción"])))

RMSE para modelo AR:  3560.509742675954


In [82]:
##Graficando el modelo para los recuperados
Recuperados_Bog=go.Figure()
Recuperados_Bog.add_trace(go.Scatter(x=model_train_Bog_recu.index, y=model_train_Bog_recu["Recuperados"],
                    mode='lines+markers',name="Datos de entrenamiento para casos recuperados"))
Recuperados_Bog.add_trace(go.Scatter(x=valid_Bog_recu.index, y=valid_Bog_recu["Recuperados"],
                    mode='lines+markers',name="Datos de Validación para casos recuperados",))
Recuperados_Bog.add_trace(go.Scatter(x=valid_Bog_recu.index, y=y_pred_Bog_recu["AR Predicción"],
                    mode='lines+markers',name="Predicción de casos recuperados",))
Recuperados_Bog.update_layout(title="Predicción casos recuperados modelo AR",
                 xaxis_title="Fecha",yaxis_title="Casos  recuperados",legend=dict(x=0,y=1,traceorder="normal"))
Recuperados_Bog.show()

In [83]:
#Crea datos de predicción a 30 días
Prediccion_AR_Bog_recu=[]
for i in range(1,30):
    Prediccion_AR_Bog_recu.append(model_ar_Bog_recu.predict(len(valid_Bog_recu)+i)[-1])

##Muertes

In [84]:
##Entrenamiento
model_train_Bog_M=df_agrupado2_Bog.iloc[:int(df_agrupado2_Bog.shape[0]*0.90)]
valid_Bog_M=df_agrupado2_Bog.iloc[int(df_agrupado2_Bog.shape[0]*0.90):]
y_pred_Bog_M=valid_Bog_M.copy()

In [85]:
##Selección
model_sarima_Bog_M= auto_arima(model_train_Bog_M["Muertos"],trace=True, error_action='ignore', 
                         start_p=0,start_q=0,max_p=2,max_q=2,m=9,
                   suppress_warnings=True,stepwise=True,seasonal=True)
model_sarima_Bog_M.fit(model_train_Bog_M["Muertos"])

Fit ARIMA: order=(0, 1, 0) seasonal_order=(1, 0, 1, 9); AIC=1635.801, BIC=1648.026, Fit time=0.347 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 0, 0, 9); AIC=1639.390, BIC=1645.502, Fit time=0.014 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 0, 0, 9); AIC=1599.762, BIC=1611.987, Fit time=0.286 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 0, 1, 9); AIC=1548.974, BIC=1561.199, Fit time=0.230 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 0, 1, 9); AIC=1550.974, BIC=1566.255, Fit time=0.423 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 0, 0, 9); AIC=1551.708, BIC=1560.877, Fit time=0.058 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 0, 2, 9); AIC=1550.974, BIC=1566.255, Fit time=0.562 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 0, 2, 9); AIC=1552.974, BIC=1571.311, Fit time=0.767 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 0, 1, 9); AIC=1550.772, BIC=1566.053, Fit time=0.296 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_o

ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(2, 1, 2),
      out_of_sample_size=0, scoring='mse', scoring_args={},
      seasonal_order=(0, 0, 0, 9), solver='lbfgs', start_params=None,
      suppress_warnings=True, transparams=True, trend='c')

In [86]:
#Predicción
prediction_sarima_Bog_M=model_sarima_Bog_M.predict(len(valid_Bog_M))
y_pred_Bog_M["SARIMA Model Prediction"]=prediction_sarima_Bog_M

In [87]:
#Métricas RMSE
model_scores_Bog_M=[]
model_scores_Bog_M.append(np.sqrt(mean_squared_error(y_pred_Bog_M["Muertos"],y_pred_Bog_M["SARIMA Model Prediction"])))
print("RMSE para modelo SARIMA: ",np.sqrt(mean_squared_error(y_pred_Bog_M["Muertos"],y_pred_Bog_M["SARIMA Model Prediction"])))

RMSE para modelo SARIMA:  37.373834240470764


In [88]:
#Gráfica SARIMA Muertes
Muertes_Bog=go.Figure()
Muertes_Bog.add_trace(go.Scatter(x=model_train_Bog_M.index, y=model_train_Bog_M["Muertos"],
                    mode='lines+markers',name="Datos de entrenamiento para muertes"))
Muertes_Bog.add_trace(go.Scatter(x=valid_Bog_M.index, y=valid_Bog_M["Muertos"],
                    mode='lines+markers',name="Datos de validación para muertes",))
Muertes_Bog.add_trace(go.Scatter(x=valid_Bog_M.index, y=y_pred_Bog_M["SARIMA Model Prediction"],
                    mode='lines+markers',name="Predicción para muertes",))
Muertes_Bog.update_layout(title="Predicción muertes modelo SARIMA",
                 xaxis_title="Fecha",yaxis_title="Muertes",legend=dict(x=0,y=1,traceorder="normal"))
Muertes_Bog.show()

In [89]:
#Se crean datos de predicción a 30 días
Prediccion_SARIMA_Bog_M=[]
for i in range(1,30):
    Prediccion_SARIMA_Bog_M.append(model_sarima_Bog_M.predict(len(valid_Bog_M)+i)[-1])

#Resumen general predicción casos confirmados, nuevos, activos, recuperados y muertes Bogotá

In [90]:
##Se tiene en cuenta que dada la reapertura se tiene una expectativa de crecimiento de los nuevos casospor lo cual se consideran los modelos evaluados pensando en ese escenario

pd.set_option('display.float_format', lambda x: '%.6f' % x)
Predicciones_Bog=zip(nuevo_dia,Prediccion_AR_Bog_Con,Prediccion_AR_Bog_recu,Prediccion_SARIMA_Bog_M )
Predicciones_Bog=pd.DataFrame(list(Predicciones_Bog), columns=["Fechas","Modelo AR Confirmados","Modelo AR Recuperados", "Modelo SARIMA Muertes"])
Predicciones_Bog.head()

,Fechas,Modelo AR Confirmados,Modelo AR Recuperados,Modelo SARIMA Muertes
0,2020-09-02,228997.898264,4864.329885,29.912633
1,2020-09-03,232603.629565,4892.206646,30.052135
2,2020-09-04,236240.152429,4920.087473,30.192066
3,2020-09-05,239887.861025,4947.867164,30.332236
4,2020-09-06,243555.848751,4975.624412,30.472538


In [91]:
##Predicciones 1ra semana
Predicciones_Bog["Activos"]= Predicciones_Bog["Modelo AR Confirmados"]-Predicciones_Bog["Modelo AR Recuperados"]-Predicciones_Bog["Modelo SARIMA Muertes"]
Predicciones_Bog["Nuevos"]= Predicciones_Bog["Modelo AR Confirmados"].diff()

Predicciones_Bog.head(7)

,Fechas,Modelo AR Confirmados,Modelo AR Recuperados,Modelo SARIMA Muertes,Activos,Nuevos
0,2020-09-02,228997.898264,4864.329885,29.912633,224103.655747,nan
1,2020-09-03,232603.629565,4892.206646,30.052135,227681.370784,3605.731301
2,2020-09-04,236240.152429,4920.087473,30.192066,231289.872889,3636.522864
3,2020-09-05,239887.861025,4947.867164,30.332236,234909.661625,3647.708597
4,2020-09-06,243555.848751,4975.624412,30.472538,238549.751801,3667.987726
5,2020-09-07,247247.300617,5003.445093,30.612914,242213.242610,3691.451866
6,2020-09-08,250960.609246,5031.279071,30.753330,245898.576845,3713.308629


In [92]:
##Predicciones cuarta semana
Predicciones_Bog.tail(7)

,Fechas,Modelo AR Confirmados,Modelo AR Recuperados,Modelo SARIMA Muertes,Activos,Nuevos
22,2020-09-24,313147.645594,5476.169899,33.000736,307638.474959,4041.377299
23,2020-09-25,317209.647406,5503.976603,33.141202,311672.529601,4062.001812
24,2020-09-26,321292.276624,5531.783005,33.281669,315727.211949,4082.629218
25,2020-09-27,325395.514797,5559.589448,33.422136,319802.503213,4103.238173
26,2020-09-28,329519.367932,5587.396066,33.562603,323898.409263,4123.853135
27,2020-09-29,333663.842028,5615.202677,33.703070,328014.936281,4144.474095
28,2020-09-30,337828.934582,5643.009186,33.843536,332152.081860,4165.092554


##Intervalos de confianza

In [93]:
Int_Conf = sms.DescrStatsW(y_pred_Bog_Con["AR Predicción"]).tconfint_mean()
Int_Recu = sms.DescrStatsW(y_pred_Bog_recu["AR Predicción"]).tconfint_mean()
Int_Muertos = sms.DescrStatsW(y_pred_Bog_M["SARIMA Model Prediction"]).tconfint_mean()

In [94]:
pd.set_option('display.float_format', lambda x: '%.6f' % x)
Intervalos_Bog=zip(Int_Conf,Int_Recu,Int_Muertos)
Intervalos_Bog=pd.DataFrame(list(Intervalos_Bog), columns=["Int_Confirmados","Int_Recuperados", "Int_Muertes"])
Intervalos_Bog

,Int_Confirmados,Int_Recuperados,Int_Muertes
0,203993.101971,4398.479110,29.787834
1,218691.842713,4579.641431,42.649354


##Métricas

In [95]:
Error_Bog=zip(model_scores_Bog_Conf,model_scores_Bog_recu,model_scores_Bog_M)
Error_Bog=pd.DataFrame(list(Error_Bog), columns=["RMSE_Confirmados","RMSE_Recuperados", "RMSE_Muertes"])
Error_Bog

,RMSE_Confirmados,RMSE_Recuperados,RMSE_Muertes
0,4299.557647,3560.509743,37.373834


#Ciudad de Medellín

In [96]:
##Se filtran los datos para la ciudad de Medellín
df_agrupado_Med = new_df[new_df['Ciudad_de_ubicación'] == 'Medellín']
df_agrupado_Med.head()

,Fecha_diagnostico,atención,Ciudad_de_ubicación,Confirmados,Recuperados,Muertos,Activos
2,2020-03-09,RECUPERADOS,Medellín,1,1,0,0
3,2020-03-11,RECUPERADOS,Medellín,1,1,0,0
4,2020-03-11,RECUPERADOS,Medellín,1,1,0,0
19,2020-03-14,RECUPERADOS,Medellín,1,1,0,0
20,2020-03-14,RECUPERADOS,Medellín,1,1,0,0


In [97]:
df_agrupado2_Med= df_agrupado_Med.groupby(["Fecha_diagnostico"]).agg({"Confirmados":'sum',"Recuperados":'sum',"Activos":'sum',"Muertos":'sum'})
df_agrupado2_Med.head()

,Confirmados,Recuperados,Activos,Muertos
Fecha_diagnostico,,,,
2020-03-09,1,1,0,0
2020-03-11,2,2,0,0
2020-03-14,2,2,0,0
2020-03-15,1,1,0,0
2020-03-19,3,3,0,0


In [98]:
##Se crean las columnas con la frecuencia acumulada por tipo de casos

df_agrupado2_Med["Confirmados_acum"] =df_agrupado2_Med["Confirmados"].cumsum()
df_agrupado2_Med["Recuperados_acum"] =df_agrupado2_Med["Recuperados"].cumsum()
df_agrupado2_Med["Activos_acum"] =df_agrupado2_Med["Activos"].cumsum()
df_agrupado2_Med["Muertos_acum"] =df_agrupado2_Med["Muertos"].cumsum()
df_agrupado2_Med.head()

,Confirmados,Recuperados,Activos,Muertos,Confirmados_acum,Recuperados_acum,Activos_acum,Muertos_acum
Fecha_diagnostico,,,,,,,,
2020-03-09,1,1,0,0,1,1,0,0
2020-03-11,2,2,0,0,3,3,0,0
2020-03-14,2,2,0,0,5,5,0,0
2020-03-15,1,1,0,0,6,6,0,0
2020-03-19,3,3,0,0,9,9,0,0


In [99]:
df_agrupado2_Med["Días"]=df_agrupado2_Med.index-df_agrupado2_Med.index.min()
df_agrupado2_Med["Días"]=df_agrupado2_Med["Días"].dt.days

In [100]:
df_agrupado2_Med.tail(10)

,Confirmados,Recuperados,Activos,Muertos,Confirmados_acum,Recuperados_acum,Activos_acum,Muertos_acum,Días
Fecha_diagnostico,,,,,,,,,
2020-08-27,719,20,692,7,45755,40028,4735,992,171
2020-08-28,710,9,694,7,46465,40037,5429,999,172
2020-08-29,548,11,535,2,47013,40048,5964,1001,173
2020-08-30,402,5,397,0,47415,40053,6361,1001,174
2020-08-31,608,5,597,6,48023,40058,6958,1007,175
2020-09-01,502,1,496,5,48525,40059,7454,1012,176
2020-09-02,890,0,886,4,49415,40059,8340,1016,177
2020-09-03,578,0,577,1,49993,40059,8917,1017,178
2020-09-04,609,0,609,0,50602,40059,9526,1017,179


In [101]:
##Se eliminan los últimos 4 días debido a que se observan retrasos en la información de recuperados y muertes
df_agrupado2_Med = df_agrupado2_Med[:-4]

In [102]:
df_agrupado2_Med.tail(10)

,Confirmados,Recuperados,Activos,Muertos,Confirmados_acum,Recuperados_acum,Activos_acum,Muertos_acum,Días
Fecha_diagnostico,,,,,,,,,
2020-08-23,578,285,289,4,42664,39065,2645,954,167
2020-08-24,702,291,402,9,43366,39356,3047,963,168
2020-08-25,885,370,507,8,44251,39726,3554,971,169
2020-08-26,785,282,489,14,45036,40008,4043,985,170
2020-08-27,719,20,692,7,45755,40028,4735,992,171
2020-08-28,710,9,694,7,46465,40037,5429,999,172
2020-08-29,548,11,535,2,47013,40048,5964,1001,173
2020-08-30,402,5,397,0,47415,40053,6361,1001,174
2020-08-31,608,5,597,6,48023,40058,6958,1007,175


#Resumen inicial estado actual Medellín

In [103]:
##Se presenta un resumen general acumulado del estado actual de casos

print("Informacion General Medellín ")
print("Número total de casos confirmados en Medellín: ",df_agrupado2_Med["Confirmados_acum"].iloc[-1])
print("Número total de casos recuperados en Medellín: ",df_agrupado2_Med["Recuperados_acum"].iloc[-1])
print("Número total de muertes en Medellín: ",df_agrupado2_Med["Muertos_acum"].iloc[-1])
print("Número total de casos Activos en Medellín: ",df_agrupado2_Med["Activos_acum"].iloc[-1])
print("Número aproximado de casos confirmados por día: ",np.round(df_agrupado2_Med["Confirmados_acum"].iloc[-1]/df_agrupado2_Med.shape[0]))
print("Número aproximado de casos recuperados por día: ",np.round(df_agrupado2_Med["Recuperados_acum"].iloc[-1]/df_agrupado2_Med.shape[0]))
print("Número aproximado de muertes por día: ",np.round(df_agrupado2_Med["Muertos_acum"].iloc[-1]/df_agrupado2_Med.shape[0]))
print("Nuevos casos confirmados: ",df_agrupado2_Med["Confirmados_acum"].iloc[-1]-df_agrupado2_Med["Confirmados_acum"].iloc[-2])

Informacion General Medellín 
Número total de casos confirmados en Medellín:  48525
Número total de casos recuperados en Medellín:  40059
Número total de muertes en Medellín:  1012
Número total de casos Activos en Medellín:  7454
Número aproximado de casos confirmados por día:  291.0
Número aproximado de casos recuperados por día:  240.0
Número aproximado de muertes por día:  6.0
Nuevos casos confirmados:  502


In [104]:
##Se grafica el comportamiento del crecimiento semanal por tipo de casos

df_agrupado2_Med["Semana"]=df_agrupado2_Med.index.weekofyear

semana_num=[]
semana_confirmados=[]
semana_recuperados=[]
semana_muertos=[]
w=1
for i in list(df_agrupado2_Med["Semana"].unique()):
    semana_confirmados.append(df_agrupado2_Med[df_agrupado2_Med["Semana"]==i]["Confirmados_acum"].iloc[-1])
    semana_recuperados.append(df_agrupado2_Med[df_agrupado2_Med["Semana"]==i]["Recuperados_acum"].iloc[-1])
    semana_muertos.append(df_agrupado2_Med[df_agrupado2_Med["Semana"]==i]["Muertos_acum"].iloc[-1])
    semana_num.append(w)
    w=w+1

crecimiento=go.Figure()
crecimiento.add_trace(go.Scatter(x=semana_num, y=semana_confirmados,
                    mode='lines+markers',
                    name='Crecimiento semanal casos confirmados'))
crecimiento.add_trace(go.Scatter(x=semana_num, y=semana_recuperados,
                    mode='lines+markers',
                    name='Crecimiento semanal casos recuperados'))
crecimiento.add_trace(go.Scatter(x=semana_num, y=semana_muertos,
                    mode='lines+markers',
                    name='Crecimiento semanal muertos'))
crecimiento.update_layout(title="Crecimiento semanal por tipo de casos en Medellín",
                 xaxis_title="Semana",yaxis_title="Número de casos",legend=dict(x=0,y=1,traceorder="normal"))
crecimiento.show()

In [105]:
##Se calcula las tasas de mortalidad y recuperación
df_agrupado2_Med["Tasa de mortalidad"]=(df_agrupado2_Med["Muertos_acum"]/df_agrupado2_Med["Confirmados_acum"])*100
df_agrupado2_Med["Tasa de recuperación"]=(df_agrupado2_Med["Recuperados_acum"]/df_agrupado2_Med["Confirmados_acum"])*100
df_agrupado2_Med["Tasa de crecimiento"]=(df_agrupado2_Med["Confirmados"]/df_agrupado2_Med["Confirmados_acum"])*100
print("Tasa de mortalidad promedio",df_agrupado2_Med["Tasa de mortalidad"].mean())
print("Tasa de recuperación promedio",df_agrupado2_Med["Tasa de recuperación"].mean())


Tasa de mortalidad promedio 1.438083272353736
Tasa de recuperación promedio 97.69994646769045


In [106]:
print("Se presenta la predicción de muertes en la ciudad de Medellín, cuya tasa de mortalidad promedio es:",df_agrupado2_Med["Tasa de mortalidad"].mean())
print("Se presenta la predicción de casos recuperados en la ciudad de Medellín, cuya tasa de recuperación promedio es:",df_agrupado2_Med["Tasa de recuperación"].mean())
print("Se presenta el crecimiento de los casos confirmados en la ciudad de Medellín, cuya tasa de crecimiento promedio es:",df_agrupado2_Med["Tasa de crecimiento"].mean())

Se presenta la predicción de muertes en la ciudad de Medellín, cuya tasa de mortalidad promedio es: 1.438083272353736
Se presenta la predicción de casos recuperados en la ciudad de Medellín, cuya tasa de recuperación promedio es: 97.69994646769045
Se presenta el crecimiento de los casos confirmados en la ciudad de Medellín, cuya tasa de crecimiento promedio es: 6.366455036216248


In [107]:
##Se grafica el comportamiento de las tasas
Tasas = make_subplots(rows=2, cols=1,
                   subplot_titles=("Tasa de recuperación", "Tasa de mortalidad"))
Tasas.add_trace(
    go.Scatter(x=df_agrupado2_Med.index, y=(df_agrupado2_Med["Recuperados_acum"]/df_agrupado2_Med["Confirmados_acum"])*100,name="Tasa de recuperación"),
    row=1, col=1
)
Tasas.add_trace(
    go.Scatter(x=df_agrupado2_Med.index, y=(df_agrupado2_Med["Muertos_acum"]/df_agrupado2_Med["Confirmados_acum"])*100,name="Tasa de mortalidad"),
    row=2, col=1
)
Tasas.update_layout(height=1000,legend=dict(x=-0.1,y=1.2,traceorder="normal"))
Tasas.update_xaxes(title_text="Fecha", row=1, col=1)
Tasas.update_yaxes(title_text="tasa de recuperación", row=1, col=1)
Tasas.update_xaxes(title_text="Fecha", row=1, col=2)
Tasas.update_yaxes(title_text="Tasa de mortalidad", row=1, col=2)
Tasas.show()

In [108]:
##Se grafica el factor de crecimiento de los tipos de casos
Factor_Crecimiento=go.Figure()
Factor_Crecimiento.add_trace(go.Scatter(x=df_agrupado2_Med.index, y=df_agrupado2_Med["Confirmados"]/df_agrupado2_Med["Confirmados"].shift(),
                    mode='lines',
                    name='Factor de crecimiento casos confirmados'))
Factor_Crecimiento.add_trace(go.Scatter(x=df_agrupado2_Med.index, y=df_agrupado2_Med["Recuperados"]/df_agrupado2_Med["Recuperados"].shift(),
                    mode='lines',
                    name='Factor de crecimiento casos recuperados'))
Factor_Crecimiento.add_trace(go.Scatter(x=df_agrupado2_Med.index, y=df_agrupado2_Med["Muertos"]/df_agrupado2_Med["Muertos"].shift(),
                    mode='lines',
                    name='Factor de crecimiento muertes'))
Factor_Crecimiento.update_layout(title="Factor de crecimiento casos confirmados, recuperados y muertes",
                 xaxis_title="Fecha",yaxis_title="Factor de crecimiento",
                 legend=dict(x=0,y=-0.4,traceorder="normal"))
Factor_Crecimiento.show()

#Modelo de predicción AR para Medellín

##Casos confirmados





In [109]:
##Entrenamiento 
model_train_Med_Con=df_agrupado2_Med.iloc[:int(df_agrupado2_Med.shape[0]*0.95)]
valid_Med_Con=df_agrupado2_Med.iloc[int(df_agrupado2_Med.shape[0]*0.95):]
y_pred_Med_Con=valid_Med_Con.copy()

In [110]:
##Selección del modelo
model_ar_Med_Con= auto_arima(model_train_Med_Con["Confirmados_acum"],trace=True, error_action='ignore', start_p=0,start_q=0,max_p=5,max_q=0,
                   suppress_warnings=True,stepwise=False,seasonal=False)
model_ar_Med_Con.fit(model_train_Med_Con["Confirmados_acum"])

Fit ARIMA: order=(0, 2, 0); AIC=1975.242, BIC=1981.342, Fit time=0.008 seconds
Fit ARIMA: order=(1, 2, 0); AIC=1950.769, BIC=1959.918, Fit time=0.040 seconds
Fit ARIMA: order=(2, 2, 0); AIC=1944.915, BIC=1957.114, Fit time=0.059 seconds
Fit ARIMA: order=(3, 2, 0); AIC=1946.795, BIC=1962.044, Fit time=0.087 seconds
Fit ARIMA: order=(4, 2, 0); AIC=1927.529, BIC=1945.828, Fit time=0.130 seconds
Fit ARIMA: order=(5, 2, 0); AIC=1929.480, BIC=1950.829, Fit time=0.214 seconds
Total fit time: 0.547 seconds


ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(4, 2, 0),
      out_of_sample_size=0, scoring='mse', scoring_args={}, seasonal_order=None,
      solver='lbfgs', start_params=None, suppress_warnings=True,
      transparams=True, trend='c')

In [111]:
##Calculo predicción
prediction_ar_Med_Con=model_ar_Med_Con.predict(len(valid_Med_Con))
y_pred_Med_Con["AR Predicción"]=prediction_ar_Med_Con

In [112]:
##Métrica RMSE
model_scores_Med_Conf=[]
model_scores_Med_Conf.append(np.sqrt(mean_squared_error(y_pred_Med_Con["Confirmados_acum"],y_pred_Med_Con["AR Predicción"])))
print("RMSE para modelo AR: ",np.sqrt(mean_squared_error(y_pred_Med_Con["Confirmados_acum"],y_pred_Med_Con["AR Predicción"])))

RMSE para modelo AR:  354.3294688809771


In [113]:
##Graficando el modelo para los confirmados
Confirmados_Med=go.Figure()
Confirmados_Med.add_trace(go.Scatter(x=model_train_Med_Con.index, y=model_train_Med_Con["Confirmados_acum"],
                    mode='lines+markers',name="Datos de entrenamiento para casos confirmados"))
Confirmados_Med.add_trace(go.Scatter(x=valid_Med_Con.index, y=valid_Med_Con["Confirmados_acum"],
                    mode='lines+markers',name="Datos de Validación para casos confirmados",))
Confirmados_Med.add_trace(go.Scatter(x=valid_Med_Con.index, y=y_pred_Med_Con["AR Predicción"],
                    mode='lines+markers',name="Predicción de casos confirmados",))
Confirmados_Med.update_layout(title="Predicción casos confirmados modelo AR",
                 xaxis_title="Fecha",yaxis_title="Casos  confirmados",legend=dict(x=0,y=1,traceorder="normal"))
Confirmados_Med.show()

In [114]:
#Crea datos de predicción a 30 días
nuevo_dia=[]
Prediccion_AR_Med_Con=[]
for i in range(1,30):
    nuevo_dia.append(df_agrupado2_Med.index[-1]+timedelta(days=i))
    Prediccion_AR_Med_Con.append(model_ar_Med_Con.predict(len(valid_Med_Con)+i)[-1])

##Casos recuperados

In [115]:
##Se debe considerar que se observan retrasos en el reporte de casos al mostrar de forma consecutiva 0 recuperados y 0 muertes, teniendo en cuenta que aún hay casos activos en la región y que los casos no permanecen activos indefinidamente 
##Entrenamiento
model_train_Med_recu=df_agrupado2_Med.iloc[:int(df_agrupado2_Med.shape[0]*0.95)]
valid_Med_recu=df_agrupado2_Med.iloc[int(df_agrupado2_Med.shape[0]*0.95):]
y_pred_Med_recu=valid_Med_recu.copy()

In [116]:
##Selección
model_ar_Med_recu= auto_arima(model_train_Med_recu["Recuperados"],trace=True, error_action='ignore', start_p=0,start_q=0,max_p=10,max_q=0,
                   suppress_warnings=True,stepwise=False,seasonal=False)
model_ar_Med_recu.fit(model_train_Med_recu["Recuperados"])

Fit ARIMA: order=(0, 1, 0); AIC=1981.039, BIC=1987.151, Fit time=0.005 seconds
Fit ARIMA: order=(1, 1, 0); AIC=1956.982, BIC=1966.151, Fit time=0.047 seconds
Fit ARIMA: order=(2, 1, 0); AIC=1951.622, BIC=1963.847, Fit time=0.052 seconds
Fit ARIMA: order=(3, 1, 0); AIC=1952.853, BIC=1968.134, Fit time=0.085 seconds
Fit ARIMA: order=(4, 1, 0); AIC=1939.663, BIC=1958.001, Fit time=0.145 seconds
Fit ARIMA: order=(5, 1, 0); AIC=1941.162, BIC=1962.555, Fit time=0.210 seconds
Fit ARIMA: order=(6, 1, 0); AIC=1942.938, BIC=1967.388, Fit time=0.377 seconds
Fit ARIMA: order=(7, 1, 0); AIC=1944.566, BIC=1972.073, Fit time=0.955 seconds
Fit ARIMA: order=(8, 1, 0); AIC=1943.076, BIC=1973.638, Fit time=1.207 seconds
Fit ARIMA: order=(9, 1, 0); AIC=1940.675, BIC=1974.294, Fit time=2.205 seconds
Fit ARIMA: order=(10, 1, 0); AIC=1941.041, BIC=1977.716, Fit time=3.570 seconds
Total fit time: 8.868 seconds


ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(4, 1, 0),
      out_of_sample_size=0, scoring='mse', scoring_args={}, seasonal_order=None,
      solver='lbfgs', start_params=None, suppress_warnings=True,
      transparams=True, trend='c')

In [117]:
##Predicción
prediction_ar_Med_recu=model_ar_Med_recu.predict(len(valid_Med_recu))
y_pred_Med_recu["AR Predicción"]=prediction_ar_Med_recu

In [118]:
#Métrica RMSE
model_scores_Med_recu=[]
model_scores_Med_recu.append(np.sqrt(mean_squared_error(y_pred_Med_recu["Recuperados"],y_pred_Med_recu["AR Predicción"])))
print("RMSE para modelo AR: ",np.sqrt(mean_squared_error(y_pred_Med_recu["Recuperados"],y_pred_Med_recu["AR Predicción"])))

RMSE para modelo AR:  292.7366976921789


In [119]:
##Graficando el modelo para los recuperados
Recuperados_Med=go.Figure()
Recuperados_Med.add_trace(go.Scatter(x=model_train_Med_recu.index, y=model_train_Med_recu["Recuperados"],
                    mode='lines+markers',name="Datos de entrenamiento para casos recuperados"))
Recuperados_Med.add_trace(go.Scatter(x=valid_Med_recu.index, y=valid_Med_recu["Recuperados"],
                    mode='lines+markers',name="Datos de Validación para casos recuperados",))
Recuperados_Med.add_trace(go.Scatter(x=valid_Med_recu.index, y=y_pred_Med_recu["AR Predicción"],
                    mode='lines+markers',name="Predicción de casos recuperados",))
Recuperados_Med.update_layout(title="Predicción casos recuperados modelo AR",
                 xaxis_title="Fecha",yaxis_title="Casos  recuperados",legend=dict(x=0,y=1,traceorder="normal"))
Recuperados_Med.show()

In [120]:
#Crea datos de predicción a 30 días
Prediccion_AR_Med_recu=[]
for i in range(1,30):
    Prediccion_AR_Med_recu.append(model_ar_Med_recu.predict(len(valid_Med_recu)+i)[-1])

##Muertes

In [121]:
##Entrenamiento
model_train_Med_M=df_agrupado2_Med.iloc[:int(df_agrupado2_Med.shape[0]*0.90)]
valid_Med_M=df_agrupado2_Med.iloc[int(df_agrupado2_Med.shape[0]*0.90):]
y_pred_Med_M=valid_Med_M.copy()

In [122]:
##Selección
model_sarima_Med_M= auto_arima(model_train_Med_M["Muertos"],trace=True, error_action='ignore', 
                         start_p=0,start_q=0,max_p=3,max_q=3,m=9,
                   suppress_warnings=True,stepwise=True,seasonal=True)
model_sarima_Med_M.fit(model_train_Med_M["Muertos"])

Fit ARIMA: order=(0, 1, 0) seasonal_order=(1, 0, 1, 9); AIC=885.835, BIC=897.851, Fit time=0.229 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 0, 0, 9); AIC=888.070, BIC=894.078, Fit time=0.012 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 0, 0, 9); AIC=863.059, BIC=875.075, Fit time=0.165 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 0, 1, 9); AIC=831.706, BIC=843.722, Fit time=0.136 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 0, 1, 9); AIC=833.460, BIC=848.480, Fit time=0.228 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 0, 0, 9); AIC=831.796, BIC=840.807, Fit time=0.047 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 0, 2, 9); AIC=832.534, BIC=847.554, Fit time=0.399 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 0, 2, 9); AIC=832.340, BIC=850.364, Fit time=0.698 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 0, 1, 9); AIC=833.404, BIC=848.424, Fit time=0.215 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 0, 1, 9);

ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(0, 1, 1),
      out_of_sample_size=0, scoring='mse', scoring_args={},
      seasonal_order=(0, 0, 1, 9), solver='lbfgs', start_params=None,
      suppress_warnings=True, transparams=True, trend='c')

In [123]:
#Predicción
prediction_sarima_Med_M=model_sarima_Med_M.predict(len(valid_Med_M))
y_pred_Med_M["SARIMA Model Prediction"]=prediction_sarima_Med_M

In [124]:
#Métricas RMSE
model_scores_Med_M=[]
model_scores_Med_M.append(np.sqrt(mean_squared_error(y_pred_Med_M["Muertos"],y_pred_Med_M["SARIMA Model Prediction"])))
print("RMSE para modelo SARIMA: ",np.sqrt(mean_squared_error(y_pred_Med_M["Muertos"],y_pred_Med_M["SARIMA Model Prediction"])))

RMSE para modelo SARIMA:  16.18369570731644


In [125]:
#Gráfica SARIMA Muertes
Muertes_Med=go.Figure()
Muertes_Med.add_trace(go.Scatter(x=model_train_Med_M.index, y=model_train_Med_M["Muertos"],
                    mode='lines+markers',name="Datos de entrenamiento para muertes"))
Muertes_Med.add_trace(go.Scatter(x=valid_Med_M.index, y=valid_Med_M["Muertos"],
                    mode='lines+markers',name="Datos de validación para muertes",))
Muertes_Med.add_trace(go.Scatter(x=valid_Med_M.index, y=y_pred_Med_M["SARIMA Model Prediction"],
                    mode='lines+markers',name="Predicción para muertes",))
Muertes_Med.update_layout(title="Predicción muertes modelo SARIMA",
                 xaxis_title="Fecha",yaxis_title="Muertes",legend=dict(x=0,y=1,traceorder="normal"))
Muertes_Med.show()

In [126]:
#Se crean datos de predicción a 30 días
Prediccion_SARIMA_Med_M=[]
for i in range(1,30):
    Prediccion_SARIMA_Med_M.append(model_sarima_Med_M.predict(len(valid_Med_M)+i)[-1])

#Resumen general predicción casos confirmados, nuevos, activos, recuperados y muertes Medellín

In [127]:
##Se tiene en cuenta que dada la reapertura se tiene una expectativa de crecimiento de los nuevos casospor lo cual se consideran los modelos evaluados pensando en ese escenario

pd.set_option('display.float_format', lambda x: '%.6f' % x)
Predicciones_Med=zip(nuevo_dia,Prediccion_AR_Med_Con,Prediccion_AR_Med_recu,Prediccion_SARIMA_Med_M )
Predicciones_Med=pd.DataFrame(list(Predicciones_Med), columns=["Fechas","Modelo AR Confirmados","Modelo AR Recuperados", "Modelo SARIMA Muertes"])
Predicciones_Med.head()

,Fechas,Modelo AR Confirmados,Modelo AR Recuperados,Modelo SARIMA Muertes
0,2020-09-02,50059.963263,372.468850,26.230003
1,2020-09-03,50824.346114,375.484250,26.390607
2,2020-09-04,51592.860544,375.810721,26.551211
3,2020-09-05,52360.223532,378.520614,26.711815
4,2020-09-06,53140.071217,381.730958,26.872419


In [128]:
##Predicciones 1ra semana
Predicciones_Med["Activos"]= Predicciones_Med["Modelo AR Confirmados"]-Predicciones_Med["Modelo AR Recuperados"]-Predicciones_Med["Modelo SARIMA Muertes"]
Predicciones_Med["Nuevos"]= Predicciones_Med["Modelo AR Confirmados"].diff()

In [129]:
Predicciones_Med.head(7)

,Fechas,Modelo AR Confirmados,Modelo AR Recuperados,Modelo SARIMA Muertes,Activos,Nuevos
0,2020-09-02,50059.963263,372.468850,26.230003,49661.264410,nan
1,2020-09-03,50824.346114,375.484250,26.390607,50422.471257,764.382851
2,2020-09-04,51592.860544,375.810721,26.551211,51190.498612,768.514430
3,2020-09-05,52360.223532,378.520614,26.711815,51954.991103,767.362987
4,2020-09-06,53140.071217,381.730958,26.872419,52731.467840,779.847685
5,2020-09-07,53921.361572,383.335878,27.033023,53510.992670,781.290355
6,2020-09-08,54707.474917,386.158384,27.193627,54294.122906,786.113345


In [130]:
##Predicciones cuarta semana
Predicciones_Med.tail(7)

,Fechas,Modelo AR Confirmados,Modelo AR Recuperados,Modelo SARIMA Muertes,Activos,Nuevos
22,2020-09-24,67911.642221,421.575996,29.763293,67460.302933,859.707521
23,2020-09-25,68775.904837,423.797675,29.923897,68322.183265,864.262615
24,2020-09-26,69644.704417,426.036939,30.084501,69188.582977,868.799580
25,2020-09-27,70518.182348,428.277647,30.245105,70059.659597,873.477932
26,2020-09-28,71396.174281,430.504899,30.405709,70935.263674,877.991933
27,2020-09-29,72278.786554,432.746974,30.566313,71815.473267,882.612273
28,2020-09-30,73166.001647,434.980318,30.726917,72700.294412,887.215093


##Intervalos de confianza

In [131]:
Int_Conf = sms.DescrStatsW(y_pred_Med_Con["AR Predicción"]).tconfint_mean()
Int_Recu = sms.DescrStatsW(y_pred_Med_recu["AR Predicción"]).tconfint_mean()
Int_Muertos = sms.DescrStatsW(y_pred_Med_M["SARIMA Model Prediction"]).tconfint_mean()


In [132]:
pd.set_option('display.float_format', lambda x: '%.6f' % x)
Intervalos_Med=zip(Int_Conf,Int_Recu,Int_Muertos)
Intervalos_Med=pd.DataFrame(list(Intervalos_Med), columns=["Int_Confirmados","Int_Recuperados", "Int_Muertes"])
Intervalos_Med

,Int_Confirmados,Int_Recuperados,Int_Muertes
0,44760.197678,353.235896,24.305539
1,47882.129670,372.691526,25.232756


##Métricas

In [133]:
Error_Med=zip(model_scores_Med_Conf,model_scores_Med_recu,model_scores_Med_M)
Error_Med=pd.DataFrame(list(Error_Med), columns=["RMSE_Confirmados","RMSE_Recuperados", "RMSE_Muertes"])
Error_Med

,RMSE_Confirmados,RMSE_Recuperados,RMSE_Muertes
0,354.329469,292.736698,16.183696


# Filtrado Cali

In [134]:
##Se filtran los datos para la ciudad de Cali
df_agrupado_Cal = new_df[new_df['Ciudad_de_ubicación'] == 'Cali']
df_agrupado_Cal.head()

,Fecha_diagnostico,atención,Ciudad_de_ubicación,Confirmados,Recuperados,Muertos,Activos
24,2020-03-13,RECUPERADOS,Cali,1,1,0,0
61,2020-03-16,RECUPERADOS,Cali,1,1,0,0
62,2020-03-16,RECUPERADOS,Cali,1,1,0,0
80,2020-03-17,RECUPERADOS,Cali,1,1,0,0
81,2020-03-17,RECUPERADOS,Cali,1,1,0,0


In [135]:
df_agrupado2_Cal= df_agrupado_Cal.groupby(["Fecha_diagnostico"]).agg({"Confirmados":'sum',"Recuperados":'sum',"Activos":'sum',"Muertos":'sum'})
df_agrupado2_Cal.head()

,Confirmados,Recuperados,Activos,Muertos
Fecha_diagnostico,,,,
2020-03-13,1,1,0,0
2020-03-16,2,2,0,0
2020-03-17,5,5,0,0
2020-03-20,1,1,0,0
2020-03-21,6,6,0,0


In [136]:
##Se crean las columnas con la frecuencia acumulada por tipo de casos

df_agrupado2_Cal["Confirmados_acum"] =df_agrupado2_Cal["Confirmados"].cumsum()
df_agrupado2_Cal["Recuperados_acum"] =df_agrupado2_Cal["Recuperados"].cumsum()
df_agrupado2_Cal["Activos_acum"] =df_agrupado2_Cal["Activos"].cumsum()
df_agrupado2_Cal["Muertos_acum"] =df_agrupado2_Cal["Muertos"].cumsum()
df_agrupado2_Cal.head()

,Confirmados,Recuperados,Activos,Muertos,Confirmados_acum,Recuperados_acum,Activos_acum,Muertos_acum
Fecha_diagnostico,,,,,,,,
2020-03-13,1,1,0,0,1,1,0,0
2020-03-16,2,2,0,0,3,3,0,0
2020-03-17,5,5,0,0,8,8,0,0
2020-03-20,1,1,0,0,9,9,0,0
2020-03-21,6,6,0,0,15,15,0,0


In [137]:
df_agrupado2_Cal["Días"]=df_agrupado2_Cal.index-df_agrupado2_Cal.index.min()
df_agrupado2_Cal["Días"]=df_agrupado2_Cal["Días"].dt.days


In [138]:
df_agrupado2_Cal.tail(10)

,Confirmados,Recuperados,Activos,Muertos,Confirmados_acum,Recuperados_acum,Activos_acum,Muertos_acum,Días
Fecha_diagnostico,,,,,,,,,
2020-08-27,362,23,335,4,35790,31742,2738,1310,167
2020-08-28,420,14,400,6,36210,31756,3138,1316,168
2020-08-29,241,6,226,9,36451,31762,3364,1325,169
2020-08-30,278,8,268,2,36729,31770,3632,1327,170
2020-08-31,589,5,577,7,37318,31775,4209,1334,171
2020-09-01,279,2,275,2,37597,31777,4484,1336,172
2020-09-02,409,0,404,5,38006,31777,4888,1341,173
2020-09-03,216,0,215,1,38222,31777,5103,1342,174
2020-09-04,348,0,348,0,38570,31777,5451,1342,175


In [139]:
##Se eliminan los últimos 4 días debido a que se observan retrasos en la información de recuperados y muertes
df_agrupado2_Cal = df_agrupado2_Cal[:-4]

In [140]:
df_agrupado2_Cal.tail(10)

,Confirmados,Recuperados,Activos,Muertos,Confirmados_acum,Recuperados_acum,Activos_acum,Muertos_acum,Días
Fecha_diagnostico,,,,,,,,,
2020-08-23,175,61,108,6,34146,31278,1593,1275,163
2020-08-24,486,198,277,11,34632,31476,1870,1286,164
2020-08-25,471,153,309,9,35103,31629,2179,1295,165
2020-08-26,325,90,224,11,35428,31719,2403,1306,166
2020-08-27,362,23,335,4,35790,31742,2738,1310,167
2020-08-28,420,14,400,6,36210,31756,3138,1316,168
2020-08-29,241,6,226,9,36451,31762,3364,1325,169
2020-08-30,278,8,268,2,36729,31770,3632,1327,170
2020-08-31,589,5,577,7,37318,31775,4209,1334,171


# Resumen inicial, estado actual Cali:

In [141]:
##Se presenta un resumen general acumulado del estado actual de casos

print("Informacion General Cali ")
print("Nro total de casos confirmados en Cali: ",df_agrupado2_Cal["Confirmados_acum"].iloc[-1])
print("Nro total de casos recuperados en Cali: ",df_agrupado2_Cal["Recuperados_acum"].iloc[-1])
print("Nro total de muertes en Cali: ",df_agrupado2_Cal["Muertos_acum"].iloc[-1])
print("Nro total de casos Activos en Cali: ",df_agrupado2_Cal["Activos_acum"].iloc[-1])
print("Nro aproximado de casos confirmados por día: ",np.round(df_agrupado2_Cal["Confirmados_acum"].iloc[-1]/df_agrupado2_Cal.shape[0]))
print("Nro aproximado de casos recuperados por día: ",np.round(df_agrupado2_Cal["Recuperados_acum"].iloc[-1]/df_agrupado2_Cal.shape[0]))
print("Nro aproximado de muertes por día: ",np.round(df_agrupado2_Cal["Muertos_acum"].iloc[-1]/df_agrupado2_Cal.shape[0]))
print("Nuevos casos confirmados: ",df_agrupado2_Cal["Confirmados_acum"].iloc[-1]-df_agrupado2_Cal["Confirmados_acum"].iloc[-2])


Informacion General Cali 
Nro total de casos confirmados en Cali:  37597
Nro total de casos recuperados en Cali:  31777
Nro total de muertes en Cali:  1336
Nro total de casos Activos en Cali:  4484
Nro aproximado de casos confirmados por día:  224.0
Nro aproximado de casos recuperados por día:  189.0
Nro aproximado de muertes por día:  8.0
Nuevos casos confirmados:  279


In [142]:
##Se grafica el comportamiento del crecimiento semanal por tipo de casos

df_agrupado2_Cal["Semana"]=df_agrupado2_Cal.index.weekofyear

semana_num=[]
semana_confirmados=[]
semana_recuperados=[]
semana_muertos=[]
w=1
for i in list(df_agrupado2_Cal["Semana"].unique()):
    semana_confirmados.append(df_agrupado2_Cal[df_agrupado2_Cal["Semana"]==i]["Confirmados_acum"].iloc[-1])
    semana_recuperados.append(df_agrupado2_Cal[df_agrupado2_Cal["Semana"]==i]["Recuperados_acum"].iloc[-1])
    semana_muertos.append(df_agrupado2_Cal[df_agrupado2_Cal["Semana"]==i]["Muertos_acum"].iloc[-1])
    semana_num.append(w)
    w=w+1

crecimiento=go.Figure()
crecimiento.add_trace(go.Scatter(x=semana_num, y=semana_confirmados,
                    mode='lines+markers',
                    name='Crecimiento semanal casos confirmados'))
crecimiento.add_trace(go.Scatter(x=semana_num, y=semana_recuperados,
                    mode='lines+markers',
                    name='Crecimiento semanal casos recuperados'))
crecimiento.add_trace(go.Scatter(x=semana_num, y=semana_muertos,
                    mode='lines+markers',
                    name='Crecimiento semanal muertos'))
crecimiento.update_layout(title="Crecimiento semanal por tipo de casos en Cali",
                 xaxis_title="Semana",yaxis_title="Número de casos",legend=dict(x=0,y=1,traceorder="normal"))
crecimiento.show()

In [143]:
##Se calcula las tasas de mortalidad y recuperación
df_agrupado2_Cal["Tasa de mortalidad"]=(df_agrupado2_Cal["Muertos_acum"]/df_agrupado2_Cal["Confirmados_acum"])*100
df_agrupado2_Cal["Tasa de recuperación"]=(df_agrupado2_Cal["Recuperados_acum"]/df_agrupado2_Cal["Confirmados_acum"])*100
df_agrupado2_Cal["Tasa de crecimiento"]=(df_agrupado2_Cal["Confirmados"]/df_agrupado2_Cal["Confirmados_acum"])*100
print("Tasa de mortalidad promedio",df_agrupado2_Cal["Tasa de mortalidad"].mean())
print("Tasa de recuperación promedio",df_agrupado2_Cal["Tasa de recuperación"].mean())

Tasa de mortalidad promedio 5.2356844349834475
Tasa de recuperación promedio 94.112961594055


In [144]:
print("Se presenta la predicción de muertes en la ciudad de Cali, cuya tasa de mortalidad promedio es:",df_agrupado2_Cal["Tasa de mortalidad"].mean())
print("Se presenta la predicción de casos recuperados en la ciudad de Cali, cuya tasa de recuperación promedio es:",df_agrupado2_Cal["Tasa de recuperación"].mean())
print("Se presenta el crecimiento de los casos confirmados en la ciudad de Cali, cuya tasa de crecimiento promedio es:",df_agrupado2_Cal["Tasa de crecimiento"].mean())

Se presenta la predicción de muertes en la ciudad de Cali, cuya tasa de mortalidad promedio es: 5.2356844349834475
Se presenta la predicción de casos recuperados en la ciudad de Cali, cuya tasa de recuperación promedio es: 94.112961594055
Se presenta el crecimiento de los casos confirmados en la ciudad de Cali, cuya tasa de crecimiento promedio es: 6.023243844999584


In [145]:
##Se grafica el comportamiento de las tasas
Tasas = make_subplots(rows=2, cols=1,
                   subplot_titles=("Tasa de recuperación", "Tasa de mortalidad"))
Tasas.add_trace(
    go.Scatter(x=df_agrupado2_Cal.index, y=(df_agrupado2_Cal["Recuperados_acum"]/df_agrupado2_Cal["Confirmados_acum"])*100,name="Tasa de recuperación"),
    row=1, col=1
)
Tasas.add_trace(
    go.Scatter(x=df_agrupado2_Cal.index, y=(df_agrupado2_Cal["Muertos_acum"]/df_agrupado2_Cal["Confirmados_acum"])*100,name="Tasa de mortalidad"),
    row=2, col=1
)
Tasas.update_layout(height=1000,legend=dict(x=-0.1,y=1.2,traceorder="normal"))
Tasas.update_xaxes(title_text="Fecha", row=1, col=1)
Tasas.update_yaxes(title_text="tasa de recuperación", row=1, col=1)
Tasas.update_xaxes(title_text="Fecha", row=1, col=2)
Tasas.update_yaxes(title_text="Tasa de mortalidad", row=1, col=2)
Tasas.show()


In [146]:
##Se grafica el factor de crecimiento de los tipos de casos
Factor_Crecimiento=go.Figure()
Factor_Crecimiento.add_trace(go.Scatter(x=df_agrupado2_Cal.index, y=df_agrupado2_Cal["Confirmados"]/df_agrupado2_Cal["Confirmados"].shift(),
                    mode='lines',
                    name='Factor de crecimiento casos confirmados'))
Factor_Crecimiento.add_trace(go.Scatter(x=df_agrupado2_Cal.index, y=df_agrupado2_Cal["Recuperados"]/df_agrupado2_Cal["Recuperados"].shift(),
                    mode='lines',
                    name='Factor de crecimiento casos recuperados'))
Factor_Crecimiento.add_trace(go.Scatter(x=df_agrupado2_Cal.index, y=df_agrupado2_Cal["Muertos"]/df_agrupado2_Cal["Muertos"].shift(),
                    mode='lines',
                    name='Factor de crecimiento muertes'))
Factor_Crecimiento.update_layout(title="Factor de crecimiento casos confirmados, recuperados y muertes",
                 xaxis_title="Fecha",yaxis_title="Factor de crecimiento",
                 legend=dict(x=0,y=-0.4,traceorder="normal"))
Factor_Crecimiento.show()


# Modelo de predicción AR para Cali

##Casos confirmados



In [147]:
##Entrenamiento 
model_train_Cal_Con=df_agrupado2_Cal.iloc[:int(df_agrupado2_Cal.shape[0]*0.95)]
valid_Cal_Con=df_agrupado2_Cal.iloc[int(df_agrupado2_Cal.shape[0]*0.95):]
y_pred_Cal_Con=valid_Cal_Con.copy()

In [148]:
##Selección del modelo
model_ar_Cal_Con= auto_arima(model_train_Cal_Con["Confirmados_acum"],trace=True, error_action='ignore', start_p=0,start_q=0,max_p=7,max_q=0,
                   suppress_warnings=True,stepwise=False,seasonal=False)
model_ar_Cal_Con.fit(model_train_Cal_Con["Confirmados_acum"])

Fit ARIMA: order=(0, 2, 0); AIC=2132.263, BIC=2138.376, Fit time=0.006 seconds
Fit ARIMA: order=(1, 2, 0); AIC=2111.695, BIC=2120.863, Fit time=0.035 seconds
Fit ARIMA: order=(2, 2, 0); AIC=2087.557, BIC=2099.782, Fit time=0.065 seconds
Fit ARIMA: order=(3, 2, 0); AIC=2087.064, BIC=2102.345, Fit time=0.101 seconds
Fit ARIMA: order=(4, 2, 0); AIC=2080.141, BIC=2098.479, Fit time=0.149 seconds
Fit ARIMA: order=(5, 2, 0); AIC=2060.493, BIC=2081.887, Fit time=0.299 seconds
Fit ARIMA: order=(6, 2, 0); AIC=2037.176, BIC=2061.626, Fit time=0.587 seconds
Fit ARIMA: order=(7, 2, 0); AIC=2038.062, BIC=2065.568, Fit time=1.101 seconds
Total fit time: 2.352 seconds


ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(6, 2, 0),
      out_of_sample_size=0, scoring='mse', scoring_args={}, seasonal_order=None,
      solver='lbfgs', start_params=None, suppress_warnings=True,
      transparams=True, trend='c')

In [149]:
##Calculo predicción
prediction_ar_Cal_Con=model_ar_Cal_Con.predict(len(valid_Cal_Con))
y_pred_Cal_Con["AR Predicción"]=prediction_ar_Cal_Con

In [150]:
##Métrica RMSE
model_scores_Cal_Conf=[]
model_scores_Cal_Conf.append(np.sqrt(mean_squared_error(y_pred_Cal_Con["Confirmados_acum"],y_pred_Cal_Con["AR Predicción"])))
print("RMSE para modelo AR: ",np.sqrt(mean_squared_error(y_pred_Cal_Con["Confirmados_acum"],y_pred_Cal_Con["AR Predicción"])))

RMSE para modelo AR:  1105.5807912864204


In [151]:
##Graficando el modelo para los confirmados
Confirmados_Cal=go.Figure()
Confirmados_Cal.add_trace(go.Scatter(x=model_train_Cal_Con.index, y=model_train_Cal_Con["Confirmados_acum"],
                    mode='lines+markers',name="Datos de entrenamiento para casos confirmados"))
Confirmados_Cal.add_trace(go.Scatter(x=valid_Cal_Con.index, y=valid_Cal_Con["Confirmados_acum"],
                    mode='lines+markers',name="Datos de Validación para casos confirmados",))
Confirmados_Cal.add_trace(go.Scatter(x=valid_Cal_Con.index, y=y_pred_Cal_Con["AR Predicción"],
                    mode='lines+markers',name="Predicción de casos confirmados",))
Confirmados_Cal.update_layout(title="Predicción casos confirmados modelo AR",
                 xaxis_title="Fecha",yaxis_title="Casos  confirmados",legend=dict(x=0,y=1,traceorder="normal"))
Confirmados_Cal.show()

In [152]:
#Crea datos de predicción a 30 días
nuevo_dia=[]
Prediccion_AR_Cal_Con=[]
for i in range(1,30):
    nuevo_dia.append(df_agrupado2_Cal.index[-1]+timedelta(days=i))
    Prediccion_AR_Cal_Con.append(model_ar_Cal_Con.predict(len(valid_Cal_Con)+i)[-1])

# Casos recuperados

In [153]:
##Se debe considerar que se observan retrasos en el reporte de casos al mostrar de forma consecutiva 0 recuperados y 0 muertes, teniendo en cuenta que aún hay casos activos en la región y que los casos no permanecen activos indefinidamente 
##Entrenamiento
model_train_Cal_recu=df_agrupado2_Cal.iloc[:int(df_agrupado2_Cal.shape[0]*0.95)]
valid_Cal_recu=df_agrupado2_Cal.iloc[int(df_agrupado2_Cal.shape[0]*0.95):]
y_pred_Cal_recu=valid_Cal_recu.copy()

In [154]:
##Selección
model_ar_Cal_recu= auto_arima(model_train_Cal_recu["Recuperados"],trace=True, error_action='ignore', start_p=0,start_q=0,max_p=5,max_q=0,
                   suppress_warnings=True,stepwise=False,seasonal=False)
model_ar_Cal_recu.fit(model_train_Cal_recu["Recuperados"])

Fit ARIMA: order=(0, 1, 0); AIC=2132.566, BIC=2138.691, Fit time=0.005 seconds
Fit ARIMA: order=(1, 1, 0); AIC=2110.119, BIC=2119.306, Fit time=0.029 seconds
Fit ARIMA: order=(2, 1, 0); AIC=2085.941, BIC=2098.191, Fit time=0.055 seconds
Fit ARIMA: order=(3, 1, 0); AIC=2085.581, BIC=2100.894, Fit time=0.106 seconds
Fit ARIMA: order=(4, 1, 0); AIC=2079.866, BIC=2098.242, Fit time=0.133 seconds
Fit ARIMA: order=(5, 1, 0); AIC=2063.172, BIC=2084.610, Fit time=0.284 seconds
Total fit time: 0.620 seconds


ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(5, 1, 0),
      out_of_sample_size=0, scoring='mse', scoring_args={}, seasonal_order=None,
      solver='lbfgs', start_params=None, suppress_warnings=True,
      transparams=True, trend='c')

In [155]:
##Predicción
prediction_ar_Cal_recu=model_ar_Cal_recu.predict(len(valid_Cal_recu))
y_pred_Cal_recu["AR Predicción"]=prediction_ar_Cal_recu


In [156]:
#Métrica RMSE
model_scores_Cal_recu=[]
model_scores_Cal_recu.append(np.sqrt(mean_squared_error(y_pred_Cal_recu["Recuperados"],y_pred_Cal_recu["AR Predicción"])))
print("RMSE para modelo AR: ",np.sqrt(mean_squared_error(y_pred_Cal_recu["Recuperados"],y_pred_Cal_recu["AR Predicción"])))

RMSE para modelo AR:  240.31680075409756


In [157]:
##Graficando el modelo para los recuperados
Recuperados_Cal=go.Figure()
Recuperados_Cal.add_trace(go.Scatter(x=model_train_Cal_recu.index, y=model_train_Cal_recu["Recuperados"],
                    mode='lines+markers',name="Datos de entrenamiento para casos recuperados"))
Recuperados_Cal.add_trace(go.Scatter(x=valid_Cal_recu.index, y=valid_Cal_recu["Recuperados"],
                    mode='lines+markers',name="Datos de Validación para casos recuperados",))
Recuperados_Cal.add_trace(go.Scatter(x=valid_Cal_recu.index, y=y_pred_Cal_recu["AR Predicción"],
                    mode='lines+markers',name="Predicción de casos recuperados",))
Recuperados_Cal.update_layout(title="Predicción casos recuperados modelo AR",
                 xaxis_title="Fecha",yaxis_title="Casos  recuperados",legend=dict(x=0,y=1,traceorder="normal"))
Recuperados_Cal.show()

In [158]:
#Crea datos de predicción a 30 días
Prediccion_AR_Cal_recu=[]
for i in range(1,30):
    Prediccion_AR_Cal_recu.append(model_ar_Cal_recu.predict(len(valid_Cal_recu)+i)[-1])

# Muertes

In [159]:
##Entrenamiento
model_train_Cal_M=df_agrupado2_Cal.iloc[:int(df_agrupado2_Cal.shape[0]*0.90)]
valid_Cal_M=df_agrupado2_Cal.iloc[int(df_agrupado2_Cal.shape[0]*0.90):]
y_pred_Cal_M=valid_Cal_M.copy()

In [160]:
##Selección
model_sarima_Cal_M= auto_arima(model_train_Cal_M["Muertos"],trace=True, error_action='ignore', 
                         start_p=0,start_q=0,max_p=2,max_q=2,m=7,
                   suppress_warnings=True,stepwise=True,seasonal=True)
model_sarima_Cal_M.fit(model_train_Cal_M["Muertos"])

Fit ARIMA: order=(0, 1, 0) seasonal_order=(1, 0, 1, 7); AIC=1004.005, BIC=1016.047, Fit time=0.227 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 0, 0, 7); AIC=1004.812, BIC=1010.833, Fit time=0.012 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 0, 0, 7); AIC=965.191, BIC=977.233, Fit time=0.164 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 0, 1, 7); AIC=924.781, BIC=936.823, Fit time=0.139 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 0, 1, 7); AIC=923.187, BIC=938.240, Fit time=0.616 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 0, 0, 7); AIC=924.760, BIC=936.802, Fit time=0.142 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 0, 2, 7); AIC=924.977, BIC=943.040, Fit time=1.500 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 0, 0, 7); AIC=927.903, BIC=936.935, Fit time=0.055 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(2, 0, 2, 7); AIC=924.940, BIC=946.014, Fit time=1.072 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 0, 1,

ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(0, 1, 1),
      out_of_sample_size=0, scoring='mse', scoring_args={},
      seasonal_order=(1, 0, 1, 7), solver='lbfgs', start_params=None,
      suppress_warnings=True, transparams=True, trend='c')

In [161]:
#Predicción
prediction_sarima_Cal_M=model_sarima_Cal_M.predict(len(valid_Cal_M))
y_pred_Cal_M["SARIMA Model Prediction"]=prediction_sarima_Cal_M

In [162]:
#Métricas RMSE
model_scores_Cal_M=[]
model_scores_Cal_M.append(np.sqrt(mean_squared_error(y_pred_Cal_M["Muertos"],y_pred_Cal_M["SARIMA Model Prediction"])))
print("RMSE para modelo SARIMA: ",np.sqrt(mean_squared_error(y_pred_Cal_M["Muertos"],y_pred_Cal_M["SARIMA Model Prediction"])))

RMSE para modelo SARIMA:  10.318165251129617


In [163]:
#Gráfica SARIMA Muertes
Muertes_Cal=go.Figure()
Muertes_Cal.add_trace(go.Scatter(x=model_train_Cal_M.index, y=model_train_Cal_M["Muertos"],
                    mode='lines+markers',name="Datos de entrenamiento para muertes"))
Muertes_Cal.add_trace(go.Scatter(x=valid_Cal_M.index, y=valid_Cal_M["Muertos"],
                    mode='lines+markers',name="Datos de validación para muertes",))
Muertes_Cal.add_trace(go.Scatter(x=valid_Cal_M.index, y=y_pred_Cal_M["SARIMA Model Prediction"],
                    mode='lines+markers',name="Predicción para muertes",))
Muertes_Cal.update_layout(title="Predicción muertes modelo SARIMA",
                 xaxis_title="Fecha",yaxis_title="Muertes",legend=dict(x=0,y=1,traceorder="normal"))
Muertes_Cal.show()

In [164]:
#Se crean datos de predicción a 30 días
Prediccion_SARIMA_Cal_M=[]
for i in range(1,30):
    Prediccion_SARIMA_Cal_M.append(model_sarima_Cal_M.predict(len(valid_Cal_M)+i)[-1])

#Resumen general predicción casos confirmados, nuevos, activos, recuperados y muertes Cali

In [165]:
##Se tiene en cuenta que dada la reapertura se tiene una expectativa de crecimiento de los nuevos casospor lo cual se consideran los modelos evaluados pensando en ese escenario

pd.set_option('display.float_format', lambda x: '%.6f' % x)
Predicciones_Cal=zip(nuevo_dia,Prediccion_AR_Cal_Con,Prediccion_AR_Cal_recu,Prediccion_SARIMA_Cal_M )
Predicciones_Cal=pd.DataFrame(list(Predicciones_Cal), columns=["Fechas","Modelo AR Confirmados","Modelo AR Recuperados", "Modelo SARIMA Muertes"])
Predicciones_Cal.head()

,Fechas,Modelo AR Confirmados,Modelo AR Recuperados,Modelo SARIMA Muertes
0,2020-09-02,39978.688678,288.331425,22.307655
1,2020-09-03,40613.463629,269.631522,18.551154
2,2020-09-04,41146.395656,265.551973,20.853226
3,2020-09-05,41644.352761,285.422112,17.871738
4,2020-09-06,42201.568176,293.574644,17.753507


In [166]:
##Predicciones 1ra semana
Predicciones_Cal["Activos"]= Predicciones_Cal["Modelo AR Confirmados"]-Predicciones_Cal["Modelo AR Recuperados"]-Predicciones_Cal["Modelo SARIMA Muertes"]
Predicciones_Cal["Nuevos"]= Predicciones_Cal["Modelo AR Confirmados"].diff()

Predicciones_Cal.head(7)

,Fechas,Modelo AR Confirmados,Modelo AR Recuperados,Modelo SARIMA Muertes,Activos,Nuevos
0,2020-09-02,39978.688678,288.331425,22.307655,39668.049598,nan
1,2020-09-03,40613.463629,269.631522,18.551154,40325.280953,634.774951
2,2020-09-04,41146.395656,265.551973,20.853226,40859.990456,532.932026
3,2020-09-05,41644.352761,285.422112,17.871738,41341.058911,497.957106
4,2020-09-06,42201.568176,293.574644,17.753507,41890.240025,557.215415
5,2020-09-07,42814.744314,295.511329,20.817375,42498.415609,613.176137
6,2020-09-08,43439.194292,293.776540,21.753527,43123.664225,624.449979


In [167]:
##Predicciones cuarta semana
Predicciones_Cal.tail(7)

,Fechas,Modelo AR Confirmados,Modelo AR Recuperados,Modelo SARIMA Muertes,Activos,Nuevos
22,2020-09-24,53376.347462,317.366156,20.899215,53038.082091,640.536874
23,2020-09-25,54013.723445,319.051078,22.627243,53672.045124,637.375983
24,2020-09-26,54660.660085,320.697997,20.452772,54319.509316,646.936640
25,2020-09-27,55319.634764,322.174416,20.393136,54977.067212,658.974679
26,2020-09-28,55986.201223,323.721706,22.683834,55639.795684,666.566460
27,2020-09-29,56656.219722,325.517650,23.402977,56307.299095,670.018499
28,2020-09-30,57324.718182,327.280953,24.147120,56973.290109,668.498460


#Intervalos de confianza

In [168]:
Int_Conf = sms.DescrStatsW(y_pred_Cal_Con["AR Predicción"]).tconfint_mean()
Int_Recu = sms.DescrStatsW(y_pred_Cal_recu["AR Predicción"]).tconfint_mean()
Int_Muertos = sms.DescrStatsW(y_pred_Cal_M["SARIMA Model Prediction"]).tconfint_mean()


In [169]:
pd.set_option('display.float_format', lambda x: '%.6f' % x)
Intervalos_Cal=zip(Int_Conf,Int_Recu,Int_Muertos)
Intervalos_Cal=pd.DataFrame(list(Intervalos_Cal), columns=["Int_Confirmados","Int_Recuperados", "Int_Muertes"])
Intervalos_Cal

,Int_Confirmados,Int_Recuperados,Int_Muertes
0,35951.976045,255.190225,17.711351
1,38295.366892,322.272300,19.974908


##Métricas

In [170]:
Error_Cal=zip(model_scores_Cal_Conf,model_scores_Cal_recu,model_scores_Cal_M)
Error_Cal=pd.DataFrame(list(Error_Cal), columns=["RMSE_Confirmados","RMSE_Recuperados", "RMSE_Muertes"])
Error_Cal

,RMSE_Confirmados,RMSE_Recuperados,RMSE_Muertes
0,1105.580791,240.316801,10.318165


# Filtrado Cartagena

In [171]:
##Se filtran los datos para la ciudad de Cartagena de Indias
df_agrupado_Cart = new_df[new_df['Ciudad_de_ubicación'] == 'Cartagena de Indias']
df_agrupado_Cart.head()

,Fecha_diagnostico,atención,Ciudad_de_ubicación,Confirmados,Recuperados,Muertos,Activos
6,2020-03-11,RECUPERADOS,Cartagena de Indias,1,1,0,0
25,2020-03-15,RECUPERADOS,Cartagena de Indias,1,1,0,0
26,2020-03-15,RECUPERADOS,Cartagena de Indias,1,1,0,0
57,2020-03-16,RECUPERADOS,Cartagena de Indias,1,1,0,0
72,2020-03-17,RECUPERADOS,Cartagena de Indias,1,1,0,0


In [172]:
df_agrupado2_Cart= df_agrupado_Cart.groupby(["Fecha_diagnostico"]).agg({"Confirmados":'sum',"Recuperados":'sum',"Activos":'sum',"Muertos":'sum'})
df_agrupado2_Cart.head()

,Confirmados,Recuperados,Activos,Muertos
Fecha_diagnostico,,,,
2020-03-11,1,1,0,0
2020-03-15,2,2,0,0
2020-03-16,2,1,0,1
2020-03-17,1,1,0,0
2020-03-18,3,3,0,0


In [173]:
##Se crean las columnas con la frecuencia acumulada por tipo de casos

df_agrupado2_Cart["Confirmados_acum"] =df_agrupado2_Cart["Confirmados"].cumsum()
df_agrupado2_Cart["Recuperados_acum"] =df_agrupado2_Cart["Recuperados"].cumsum()
df_agrupado2_Cart["Activos_acum"] =df_agrupado2_Cart["Activos"].cumsum()
df_agrupado2_Cart["Muertos_acum"] =df_agrupado2_Cart["Muertos"].cumsum()
df_agrupado2_Cart.head()

,Confirmados,Recuperados,Activos,Muertos,Confirmados_acum,Recuperados_acum,Activos_acum,Muertos_acum
Fecha_diagnostico,,,,,,,,
2020-03-11,1,1,0,0,1,1,0,0
2020-03-15,2,2,0,0,3,3,0,0
2020-03-16,2,1,0,1,5,4,0,1
2020-03-17,1,1,0,0,6,5,0,1
2020-03-18,3,3,0,0,9,8,0,1


In [174]:
df_agrupado2_Cart["Días"]=df_agrupado2_Cart.index-df_agrupado2_Cart.index.min()
df_agrupado2_Cart["Días"]=df_agrupado2_Cart["Días"].dt.days

In [175]:
df_agrupado2_Cart.tail(10)

,Confirmados,Recuperados,Activos,Muertos,Confirmados_acum,Recuperados_acum,Activos_acum,Muertos_acum,Días
Fecha_diagnostico,,,,,,,,,
2020-08-27,45,3,42,0,19745,18687,537,521,169
2020-08-28,113,0,113,0,19858,18687,650,521,170
2020-08-29,63,1,61,1,19921,18688,711,522,171
2020-08-30,47,2,45,0,19968,18690,756,522,172
2020-08-31,137,0,136,1,20105,18690,892,523,173
2020-09-01,47,0,47,0,20152,18690,939,523,174
2020-09-02,76,0,76,0,20228,18690,1015,523,175
2020-09-03,37,0,36,1,20265,18690,1051,524,176
2020-09-04,48,1,47,0,20313,18691,1098,524,177


In [176]:
##Se eliminan los últimos 4 días debido a que se observan retrasos en la información de recuperados y muertes
df_agrupado2_Cart = df_agrupado2_Cart[:-4]


In [177]:
df_agrupado2_Cart.tail(10)

,Confirmados,Recuperados,Activos,Muertos,Confirmados_acum,Recuperados_acum,Activos_acum,Muertos_acum,Días
Fecha_diagnostico,,,,,,,,,
2020-08-23,65,39,26,0,19004,18322,165,517,165
2020-08-24,205,180,24,1,19209,18502,189,518,166
2020-08-25,238,140,97,1,19447,18642,286,519,167
2020-08-26,253,42,209,2,19700,18684,495,521,168
2020-08-27,45,3,42,0,19745,18687,537,521,169
2020-08-28,113,0,113,0,19858,18687,650,521,170
2020-08-29,63,1,61,1,19921,18688,711,522,171
2020-08-30,47,2,45,0,19968,18690,756,522,172
2020-08-31,137,0,136,1,20105,18690,892,523,173


# Resumen inicial, estado actual Cartagena de Indias

In [178]:
##Se presenta un resumen general acumulado del estado actual de casos

print("Informacion General Cartagena ")
print("Número total de casos confirmados en Cartagena: ",df_agrupado2_Cart["Confirmados_acum"].iloc[-1])
print("Número total de casos recuperados en Cartagena: ",df_agrupado2_Cart["Recuperados_acum"].iloc[-1])
print("Número total de muertes en Cartagena: ",df_agrupado2_Cart["Muertos_acum"].iloc[-1])
print("Número total de casos Activos en Cartagena: ",df_agrupado2_Cart["Activos_acum"].iloc[-1])
print("Número aproximado de casos confirmados por día: ",np.round(df_agrupado2_Cart["Confirmados_acum"].iloc[-1]/df_agrupado2_Cart.shape[0]))
print("Número aproximado de casos recuperados por día: ",np.round(df_agrupado2_Cart["Recuperados_acum"].iloc[-1]/df_agrupado2_Cart.shape[0]))
print("Número aproximado de muertes por día: ",np.round(df_agrupado2_Cart["Muertos_acum"].iloc[-1]/df_agrupado2_Cart.shape[0]))
print("Nuevos casos confirmados: ",df_agrupado2_Cart["Confirmados_acum"].iloc[-1]-df_agrupado2_Cart["Confirmados_acum"].iloc[-2])


Informacion General Cartagena 
Número total de casos confirmados en Cartagena:  20152
Número total de casos recuperados en Cartagena:  18690
Número total de muertes en Cartagena:  523
Número total de casos Activos en Cartagena:  939
Número aproximado de casos confirmados por día:  123.0
Número aproximado de casos recuperados por día:  114.0
Número aproximado de muertes por día:  3.0
Nuevos casos confirmados:  47


In [179]:
##Se grafica el comportamiento del crecimiento semanal por tipo de casos

df_agrupado2_Cart["Semana"]=df_agrupado2_Cart.index.weekofyear

semana_num=[]
semana_confirmados=[]
semana_recuperados=[]
semana_muertos=[]
w=1
for i in list(df_agrupado2_Cart["Semana"].unique()):
    semana_confirmados.append(df_agrupado2_Cart[df_agrupado2_Cart["Semana"]==i]["Confirmados_acum"].iloc[-1])
    semana_recuperados.append(df_agrupado2_Cart[df_agrupado2_Cart["Semana"]==i]["Recuperados_acum"].iloc[-1])
    semana_muertos.append(df_agrupado2_Cart[df_agrupado2_Cart["Semana"]==i]["Muertos_acum"].iloc[-1])
    semana_num.append(w)
    w=w+1

crecimiento=go.Figure()
crecimiento.add_trace(go.Scatter(x=semana_num, y=semana_confirmados,
                    mode='lines+markers',
                    name='Crecimiento semanal casos confirmados'))
crecimiento.add_trace(go.Scatter(x=semana_num, y=semana_recuperados,
                    mode='lines+markers',
                    name='Crecimiento semanal casos recuperados'))
crecimiento.add_trace(go.Scatter(x=semana_num, y=semana_muertos,
                    mode='lines+markers',
                    name='Crecimiento semanal muertos'))
crecimiento.update_layout(title="Crecimiento semanal por tipo de casos en Cartagena de Indias",
                 xaxis_title="Semana",yaxis_title="Número de casos",legend=dict(x=0,y=1,traceorder="normal"))
crecimiento.show()

In [180]:
##Se Cartcula las tasas de mortalidad y recuperación
df_agrupado2_Cart["Tasa de mortalidad"]=(df_agrupado2_Cart["Muertos_acum"]/df_agrupado2_Cart["Confirmados_acum"])*100
df_agrupado2_Cart["Tasa de recuperación"]=(df_agrupado2_Cart["Recuperados_acum"]/df_agrupado2_Cart["Confirmados_acum"])*100
df_agrupado2_Cart["Tasa de crecimiento"]=(df_agrupado2_Cart["Confirmados"]/df_agrupado2_Cart["Confirmados_acum"])*100

print("Tasa de mortalidad promedio",df_agrupado2_Cart["Tasa de mortalidad"].mean())
print("Tasa de recuperación promedio",df_agrupado2_Cart["Tasa de recuperación"].mean())

Tasa de mortalidad promedio 6.092919924696239
Tasa de recuperación promedio 93.7082388272435


In [181]:
print("Se presenta la predicción de muertes en la ciudad de Cartagena, cuya tasa de mortalidad promedio es:",df_agrupado2_Cart["Tasa de mortalidad"].mean())
print("Se presenta la predicción de casos recuperados en la ciudad de Cartagena, cuya tasa de recuperación promedio es:",df_agrupado2_Cart["Tasa de recuperación"].mean())
print("Se presenta el crecimiento de los casos confirmados en la ciudad de Cartagena, cuya tasa de crecimiento promedio es:",df_agrupado2_Cart["Tasa de crecimiento"].mean())

Se presenta la predicción de muertes en la ciudad de Cartagena, cuya tasa de mortalidad promedio es: 6.092919924696239
Se presenta la predicción de casos recuperados en la ciudad de Cartagena, cuya tasa de recuperación promedio es: 93.7082388272435
Se presenta el crecimiento de los casos confirmados en la ciudad de Cartagena, cuya tasa de crecimiento promedio es: 6.024147962152589


In [182]:
##Se grafica el comportamiento de las tasas
Tasas = make_subplots(rows=2, cols=1,
                   subplot_titles=("Tasa de recuperación", "Tasa de mortalidad"))
Tasas.add_trace(
    go.Scatter(x=df_agrupado2_Cart.index, y=(df_agrupado2_Cart["Recuperados_acum"]/df_agrupado2_Cart["Confirmados_acum"])*100,name="Tasa de recuperación"),
    row=1, col=1
)
Tasas.add_trace(
    go.Scatter(x=df_agrupado2_Cart.index, y=(df_agrupado2_Cart["Muertos_acum"]/df_agrupado2_Cart["Confirmados_acum"])*100,name="Tasa de mortalidad"),
    row=2, col=1
)
Tasas.update_layout(height=1000,legend=dict(x=-0.1,y=1.2,traceorder="normal"))
Tasas.update_xaxes(title_text="Fecha", row=1, col=1)
Tasas.update_yaxes(title_text="Tasa de recuperación", row=1, col=1)
Tasas.update_xaxes(title_text="Fecha", row=1, col=2)
Tasas.update_yaxes(title_text="Tasa de mortalidad", row=1, col=2)
Tasas.show()

In [183]:
##Se grafica el factor de crecimiento de los tipos de casos
Factor_Crecimiento=go.Figure()
Factor_Crecimiento.add_trace(go.Scatter(x=df_agrupado2_Cart.index, y=df_agrupado2_Cart["Confirmados"]/df_agrupado2_Cart["Confirmados"].shift(),
                    mode='lines',
                    name='Factor de crecimiento casos confirmados'))
Factor_Crecimiento.add_trace(go.Scatter(x=df_agrupado2_Cart.index, y=df_agrupado2_Cart["Recuperados"]/df_agrupado2_Cart["Recuperados"].shift(),
                    mode='lines',
                    name='Factor de crecimiento casos recuperados'))
Factor_Crecimiento.add_trace(go.Scatter(x=df_agrupado2_Cart.index, y=df_agrupado2_Cart["Muertos"]/df_agrupado2_Cart["Muertos"].shift(),
                    mode='lines',
                    name='Factor de crecimiento muertes'))
Factor_Crecimiento.update_layout(title="Factor de crecimiento casos confirmados, recuperados y muertes",
                 xaxis_title="Fecha",yaxis_title="Factor de crecimiento",
                 legend=dict(x=0,y=-0.4,traceorder="normal"))
Factor_Crecimiento.show()

#Modelo de predicción AR para Cartagena de Indias

##Casos confirmados

In [184]:
##Entrenamiento 
model_train_Cart_Con=df_agrupado2_Cart.iloc[:int(df_agrupado2_Cart.shape[0]*0.95)]
valid_Cart_Con=df_agrupado2_Cart.iloc[int(df_agrupado2_Cart.shape[0]*0.95):]
y_pred_Cart_Con=valid_Cart_Con.copy()

In [185]:
##Selección del modelo
model_ar_Cart_Con= auto_arima(model_train_Cart_Con["Confirmados_acum"],trace=True, error_action='ignore', start_p=0,start_q=0,max_p=5,max_q=0,
                   suppress_warnings=True,stepwise=False,seasonal=False)
model_ar_Cart_Con.fit(model_train_Cart_Con["Confirmados_acum"])

Fit ARIMA: order=(0, 2, 0); AIC=1724.931, BIC=1730.992, Fit time=0.004 seconds
Fit ARIMA: order=(1, 2, 0); AIC=1701.708, BIC=1710.799, Fit time=0.034 seconds
Fit ARIMA: order=(2, 2, 0); AIC=1684.107, BIC=1696.229, Fit time=0.053 seconds
Fit ARIMA: order=(3, 2, 0); AIC=1684.978, BIC=1700.130, Fit time=0.089 seconds
Fit ARIMA: order=(4, 2, 0); AIC=1680.047, BIC=1698.230, Fit time=0.135 seconds
Fit ARIMA: order=(5, 2, 0); AIC=1670.855, BIC=1692.068, Fit time=0.240 seconds
Total fit time: 0.562 seconds


ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(5, 2, 0),
      out_of_sample_size=0, scoring='mse', scoring_args={}, seasonal_order=None,
      solver='lbfgs', start_params=None, suppress_warnings=True,
      transparams=True, trend='c')

In [186]:
##Cálculo predicción
prediction_ar_Cart_Con=model_ar_Cart_Con.predict(len(valid_Cart_Con))
y_pred_Cart_Con["AR Predicción"]=prediction_ar_Cart_Con

In [187]:
##Métrica RMSE
model_scores_Cart_Conf=[]
model_scores_Cart_Conf.append(np.sqrt(mean_squared_error(y_pred_Cart_Con["Confirmados_acum"],y_pred_Cart_Con["AR Predicción"])))
print("RMSE para modelo AR: ",np.sqrt(mean_squared_error(y_pred_Cart_Con["Confirmados_acum"],y_pred_Cart_Con["AR Predicción"])))

RMSE para modelo AR:  428.00754490276546


In [188]:
##Graficando el modelo para los confirmados
Confirmados_Cart=go.Figure()
Confirmados_Cart.add_trace(go.Scatter(x=model_train_Cart_Con.index, y=model_train_Cart_Con["Confirmados_acum"],
                    mode='lines+markers',name="Datos de entrenamiento para casos confirmados"))
Confirmados_Cart.add_trace(go.Scatter(x=valid_Cart_Con.index, y=valid_Cart_Con["Confirmados_acum"],
                    mode='lines+markers',name="Datos de validación para casos confirmados",))
Confirmados_Cart.add_trace(go.Scatter(x=valid_Cart_Con.index, y=y_pred_Cart_Con["AR Predicción"],
                    mode='lines+markers',name="Predicción de casos confirmados",))
Confirmados_Cart.update_layout(title="Predicción casos confirmados modelo AR",
                 xaxis_title="Fecha",yaxis_title="Casos  confirmados",legend=dict(x=0,y=1,traceorder="normal"))
Confirmados_Cart.show()

In [189]:
#Crea datos de predicción a 30 días
nuevo_dia=[]
Prediccion_AR_Cart_Con=[]
for i in range(1,30):
    nuevo_dia.append(df_agrupado2_Cart.index[-1]+timedelta(days=i))
    Prediccion_AR_Cart_Con.append(model_ar_Cart_Con.predict(len(valid_Cart_Con)+i)[-1])

##Casos recuperados

In [190]:
##Se debe considerar que se observan retrasos en el reporte de casos al mostrar de forma consecutiva 0 recuperados y 0 muertes, teniendo en cuenta que aún hay casos activos en la región y que los casos no permanecen activos indefinidamente 
##Entrenamiento
model_train_Cart_recu=df_agrupado2_Cart.iloc[:int(df_agrupado2_Cart.shape[0]*0.95)]
valid_Cart_recu=df_agrupado2_Cart.iloc[int(df_agrupado2_Cart.shape[0]*0.95):]
y_pred_Cart_recu=valid_Cart_recu.copy()

In [191]:
##Selección
model_ar_Cart_recu= auto_arima(model_train_Cart_recu["Recuperados"],trace=True, error_action='ignore', start_p=0,start_q=0,max_p=7,max_q=0,
                   suppress_warnings=True,stepwise=False,seasonal=False)
model_ar_Cart_recu.fit(model_train_Cart_recu["Recuperados"])


Fit ARIMA: order=(0, 1, 0); AIC=1728.743, BIC=1734.817, Fit time=0.005 seconds
Fit ARIMA: order=(1, 1, 0); AIC=1704.572, BIC=1713.683, Fit time=0.033 seconds
Fit ARIMA: order=(2, 1, 0); AIC=1686.960, BIC=1699.108, Fit time=0.055 seconds
Fit ARIMA: order=(3, 1, 0); AIC=1687.527, BIC=1702.711, Fit time=0.086 seconds
Fit ARIMA: order=(4, 1, 0); AIC=1683.308, BIC=1701.530, Fit time=0.144 seconds
Fit ARIMA: order=(5, 1, 0); AIC=1673.564, BIC=1694.823, Fit time=0.240 seconds
Fit ARIMA: order=(6, 1, 0); AIC=1674.293, BIC=1698.589, Fit time=0.363 seconds
Fit ARIMA: order=(7, 1, 0); AIC=1676.291, BIC=1703.624, Fit time=0.738 seconds
Total fit time: 1.673 seconds


ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(5, 1, 0),
      out_of_sample_size=0, scoring='mse', scoring_args={}, seasonal_order=None,
      solver='lbfgs', start_params=None, suppress_warnings=True,
      transparams=True, trend='c')

In [192]:
##Predicción
prediction_ar_Cart_recu=model_ar_Cart_recu.predict(len(valid_Cart_recu))
y_pred_Cart_recu["AR Predicción"]=prediction_ar_Cart_recu

In [193]:
#Métrica RMSE
model_scores_Cart_recu=[]
model_scores_Cart_recu.append(np.sqrt(mean_squared_error(y_pred_Cart_recu["Recuperados"],y_pred_Cart_recu["AR Predicción"])))
print("RMSE para modelo AR: ",np.sqrt(mean_squared_error(y_pred_Cart_recu["Recuperados"],y_pred_Cart_recu["AR Predicción"])))

RMSE para modelo AR:  70.1670080199821


In [194]:
##Graficando el modelo para los recuperados
Recuperados_Cart=go.Figure()
Recuperados_Cart.add_trace(go.Scatter(x=model_train_Cart_recu.index, y=model_train_Cart_recu["Recuperados"],
                    mode='lines+markers',name="Datos de entrenamiento para casos recuperados"))
Recuperados_Cart.add_trace(go.Scatter(x=valid_Cart_recu.index, y=valid_Cart_recu["Recuperados"],
                    mode='lines+markers',name="Datos de validación para casos recuperados",))
Recuperados_Cart.add_trace(go.Scatter(x=valid_Cart_recu.index, y=y_pred_Cart_recu["AR Predicción"],
                    mode='lines+markers',name="Predicción de casos recuperados",))
Recuperados_Cart.update_layout(title="Predicción casos recuperados modelo AR",
                 xaxis_title="Fecha",yaxis_title="Casos  recuperados",legend=dict(x=0,y=1,traceorder="normal"))
Recuperados_Cart.show()


In [195]:
#Crea datos de predicción a 30 días
Prediccion_AR_Cart_recu=[]
for i in range(1,30):
    Prediccion_AR_Cart_recu.append(model_ar_Cart_recu.predict(len(valid_Cart_recu)+i)[-1])

##Muertes

In [196]:
##Entrenamiento
model_train_Cart_M=df_agrupado2_Cart.iloc[:int(df_agrupado2_Cart.shape[0]*0.90)]
valid_Cart_M=df_agrupado2_Cart.iloc[int(df_agrupado2_Cart.shape[0]*0.90):]
y_pred_Cart_M=valid_Cart_M.copy()

In [197]:
##Selección
model_sarima_Cart_M= auto_arima(model_train_Cart_M["Muertos"],trace=True, error_action='ignore', 
                         start_p=0,start_q=0,max_p=2,max_q=2,m=7,
                   suppress_warnings=True,stepwise=True,seasonal=True)
model_sarima_Cart_M.fit(model_train_Cart_M["Muertos"])


Fit ARIMA: order=(0, 1, 0) seasonal_order=(1, 0, 1, 7); AIC=808.136, BIC=820.071, Fit time=0.238 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 0, 0, 7); AIC=806.462, BIC=812.429, Fit time=0.011 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 0, 0, 7); AIC=768.062, BIC=779.996, Fit time=0.098 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 0, 1, 7); AIC=733.841, BIC=745.776, Fit time=0.134 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 0, 1, 7); AIC=735.046, BIC=749.964, Fit time=0.224 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 0, 0, 7); AIC=732.324, BIC=741.275, Fit time=0.046 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 0, 0, 7); AIC=734.108, BIC=746.042, Fit time=0.076 seconds
Fit ARIMA: order=(0, 1, 2) seasonal_order=(0, 0, 0, 7); AIC=734.097, BIC=746.031, Fit time=0.103 seconds
Fit ARIMA: order=(1, 1, 2) seasonal_order=(0, 0, 0, 7); AIC=735.787, BIC=750.705, Fit time=0.197 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 0, 0, 7);

ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(0, 1, 1),
      out_of_sample_size=0, scoring='mse', scoring_args={},
      seasonal_order=(0, 0, 0, 7), solver='lbfgs', start_params=None,
      suppress_warnings=True, transparams=True, trend='c')

In [198]:
#Predicción
prediction_sarima_Cart_M=model_sarima_Cart_M.predict(len(valid_Cart_M))
y_pred_Cart_M["SARIMA Model Prediction"]=prediction_sarima_Cart_M

In [199]:
#Métricas RMSE
model_scores_Cart_M=[]
model_scores_Cart_M.append(np.sqrt(mean_squared_error(y_pred_Cart_M["Muertos"],y_pred_Cart_M["SARIMA Model Prediction"])))
print("RMSE para modelo SARIMA: ",np.sqrt(mean_squared_error(y_pred_Cart_M["Muertos"],y_pred_Cart_M["SARIMA Model Prediction"])))

RMSE para modelo SARIMA:  0.9409753912186837


In [200]:
#Gráfica SARIMA Muertes
Muertes_Cart=go.Figure()
Muertes_Cart.add_trace(go.Scatter(x=model_train_Cart_M.index, y=model_train_Cart_M["Muertos"],
                    mode='lines+markers',name="Datos de entrenamiento para muertes"))
Muertes_Cart.add_trace(go.Scatter(x=valid_Cart_M.index, y=valid_Cart_M["Muertos"],
                    mode='lines+markers',name="Datos de validación para muertes",))
Muertes_Cart.add_trace(go.Scatter(x=valid_Cart_M.index, y=y_pred_Cart_M["SARIMA Model Prediction"],
                    mode='lines+markers',name="Predicción para muertes",))
Muertes_Cart.update_layout(title="Predicción muertes modelo SARIMA",
                 xaxis_title="Fecha",yaxis_title="Muertes",legend=dict(x=0,y=1,traceorder="normal"))
Muertes_Cart.show()

In [201]:
#Se crean datos de predicción a 30 días
Prediccion_SARIMA_Cart_M=[]
for i in range(1,30):
    Prediccion_SARIMA_Cart_M.append(model_sarima_Cart_M.predict(len(valid_Cart_M)+i)[-1])

#Resumen general predicción casos confirmados, nuevos, activos, recuperados y muertes Cartagena de Indias

In [202]:
##Se tiene en cuenta que dada la reapertura se tiene una expectativa de crecimiento de los nuevos casospor lo cual se consideran los modelos evaluados pensando en ese escenario

pd.set_option('display.float_format', lambda x: '%.6f' % x)
Predicciones_Cart=zip(nuevo_dia,Prediccion_AR_Cart_Con,Prediccion_AR_Cart_recu,Prediccion_SARIMA_Cart_M )
Predicciones_Cart=pd.DataFrame(list(Predicciones_Cart), columns=["Fechas","Modelo AR Confirmados","Modelo AR Recuperados", "Modelo SARIMA Muertes"])
Predicciones_Cart.head()

,Fechas,Modelo AR Confirmados,Modelo AR Recuperados,Modelo SARIMA Muertes
0,2020-09-02,19771.122796,54.929039,1.252063
1,2020-09-03,19848.566265,49.474529,1.258708
2,2020-09-04,19931.529982,54.525232,1.265354
3,2020-09-05,20015.187067,54.934597,1.272000
4,2020-09-06,20094.738499,51.071103,1.278646


In [203]:
##Predicciones 1ra semana
Predicciones_Cart["Activos"]= Predicciones_Cart["Modelo AR Confirmados"]-Predicciones_Cart["Modelo AR Recuperados"]-Predicciones_Cart["Modelo SARIMA Muertes"]
Predicciones_Cart["Nuevos"]= Predicciones_Cart["Modelo AR Confirmados"].diff()

Predicciones_Cart.head(7)

,Fechas,Modelo AR Confirmados,Modelo AR Recuperados,Modelo SARIMA Muertes,Activos,Nuevos
0,2020-09-02,19771.122796,54.929039,1.252063,19714.941695,nan
1,2020-09-03,19848.566265,49.474529,1.258708,19797.833028,77.443469
2,2020-09-04,19931.529982,54.525232,1.265354,19875.739396,82.963716
3,2020-09-05,20015.187067,54.934597,1.272000,19958.980470,83.657085
4,2020-09-06,20094.738499,51.071103,1.278646,20042.388750,79.551432
5,2020-09-07,20177.439006,54.145296,1.285291,20122.008419,82.700507
6,2020-09-08,20260.671979,54.443968,1.291937,20204.936073,83.232973


In [204]:
##Predicciones cuarta semana
Predicciones_Cart.tail(7)


,Fechas,Modelo AR Confirmados,Modelo AR Recuperados,Modelo SARIMA Muertes,Activos,Nuevos
22,2020-09-24,21650.799735,58.852196,1.398269,21590.549270,90.485970
23,2020-09-25,21741.919164,59.317090,1.404915,21681.197159,91.119428
24,2020-09-26,21833.432529,59.535256,1.411561,21772.485712,91.513366
25,2020-09-27,21925.405395,59.821770,1.418207,21864.165418,91.972866
26,2020-09-28,22017.942762,60.218175,1.424852,21956.299734,92.537367
27,2020-09-29,22110.905644,60.467761,1.431498,22049.006385,92.962882
28,2020-09-30,22204.349661,60.774587,1.438144,22142.136931,93.444017


##Intervalos de confianza

In [205]:
Int_Conf = sms.DescrStatsW(y_pred_Cart_Con["AR Predicción"]).tconfint_mean()
Int_Recu = sms.DescrStatsW(y_pred_Cart_recu["AR Predicción"]).tconfint_mean()
Int_Muertos = sms.DescrStatsW(y_pred_Cart_M["SARIMA Model Prediction"]).tconfint_mean()

In [206]:
pd.set_option('display.float_format', lambda x: '%.6f' % x)
Intervalos_Cart=zip(Int_Conf,Int_Recu,Int_Muertos)
Intervalos_Cart=pd.DataFrame(list(Intervalos_Cart),  columns=["Int_Confirmados","Int_Recuperados", "Int_Muertes"])
Intervalos_Cart


,Int_Confirmados,Int_Recuperados,Int_Muertes
0,19213.031520,41.994872,1.174996
1,19540.993004,57.574630,1.209505


##Métricas

In [207]:
Error_Cart=zip(model_scores_Cart_Conf,model_scores_Cart_recu,model_scores_Cart_M)
Error_Cart=pd.DataFrame(list(Error_Cart), columns=["RMSE_Confirmados","RMSE_Recuperados", "RMSE_Muertes"])
Error_Cart

,RMSE_Confirmados,RMSE_Recuperados,RMSE_Muertes
0,428.007545,70.167008,0.940975
